In [2]:
import pandas as pd
import datetime
import csv
import pprint as pprint
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
#import sklearn.datasets as dta
#from scipy.stats import linregress
#from scipy.stats import sem
import json
import requests
import os
from dotenv import load_dotenv
import gzip
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
census_key = os.getenv("census_api_key")

# Section raw redfin data


In [13]:
# handling the large raw data file
filepath_redfin = "../data_sources/zip_code_market_tracker.tsv000"
redfinraw = pd.read_csv(filepath_redfin, delimiter = "\t")
redfinraw.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2014-08-01,2014-10-31,90,zip code,2,1920,f,Zip Code: 05489,NaN,Vermont,...,-0.325000,NaN,NaN,NaN,NaN,NaN,NaN,"Burlington, VT",15540,2024-03-10 14:36:40
1,2018-04-01,2018-06-30,90,zip code,2,18317,f,Zip Code: 43750,NaN,Ohio,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,"Cambridge, OH",15740,2024-03-10 14:36:40
2,2017-01-01,2017-03-31,90,zip code,2,13955,f,Zip Code: 33062,NaN,Florida,...,-0.600000,NaN,NaN,NaN,0.0,NaN,-0.500000,"Fort Lauderdale, FL",22744,2024-03-10 14:36:40
3,2015-12-01,2016-02-29,90,zip code,2,23759,f,Zip Code: 55437,NaN,Minnesota,...,-0.181818,NaN,NaN,NaN,0.0,-1.0,-0.166667,"Minneapolis, MN",33460,2024-03-10 14:36:40
4,2023-12-01,2024-02-29,90,zip code,2,25730,f,Zip Code: 59922,NaN,Montana,...,0.000000,NaN,NaN,NaN,1.0,1.0,1.000000,"Kalispell, MT",28060,2024-03-10 14:36:40


In [14]:
# listing the columns
cols = list(redfinraw.columns)
cols

['period_begin',
 'period_end',
 'period_duration',
 'region_type',
 'region_type_id',
 'table_id',
 'is_seasonally_adjusted',
 'region',
 'city',
 'state',
 'state_code',
 'property_type',
 'property_type_id',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'pending_sales',
 'pending_sales_mom',
 'pending_sales_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'inventory_yoy',
 'months_of_supply',
 'months_of_supply_mom',
 'months_of_supply_yoy',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'price_drops',
 'price_dro

In [24]:
#Define all zipcodes in seattle
allzipcodes = [
98102,
98103,
98107,
98109,
98112,
98115,
98117,
98119,
98125,
98133,
98177,
98105,
98106,
98108,
98112,
98116,
98118,
98122,
98126,
98134,
98136,
98144,
98146,
98178,
98199,
98101,
98104,
98113,
98121,
98141,
98154,
98161,
98165,
98170,
98174,
98175,
98181,
98185,
98190,
98191,
98194
]
# allzipcodes_str = [str(zipcode) for zipcode in allzipcodes]


In [25]:
#Change allzipcodes to match the headers in the dataframe by making them a string and appending

allzipcodes_format = []
for zipcode in allzipcodes:
    modified_zipcode = "Zip Code: " + str(zipcode)  
    allzipcodes_format.append(modified_zipcode)

print(allzipcodes_format)

['Zip Code: 98102', 'Zip Code: 98103', 'Zip Code: 98107', 'Zip Code: 98109', 'Zip Code: 98112', 'Zip Code: 98115', 'Zip Code: 98117', 'Zip Code: 98119', 'Zip Code: 98125', 'Zip Code: 98133', 'Zip Code: 98177', 'Zip Code: 98105', 'Zip Code: 98106', 'Zip Code: 98108', 'Zip Code: 98112', 'Zip Code: 98116', 'Zip Code: 98118', 'Zip Code: 98122', 'Zip Code: 98126', 'Zip Code: 98134', 'Zip Code: 98136', 'Zip Code: 98144', 'Zip Code: 98146', 'Zip Code: 98178', 'Zip Code: 98199', 'Zip Code: 98101', 'Zip Code: 98104', 'Zip Code: 98113', 'Zip Code: 98121', 'Zip Code: 98141', 'Zip Code: 98154', 'Zip Code: 98161', 'Zip Code: 98165', 'Zip Code: 98170', 'Zip Code: 98174', 'Zip Code: 98175', 'Zip Code: 98181', 'Zip Code: 98185', 'Zip Code: 98190', 'Zip Code: 98191', 'Zip Code: 98194']


In [27]:
#New Dataframe that only includes zipcodes that are in the Seattle area
metrodata = redfinraw[redfinraw["region"].isin(allzipcodes_format)]
metrodata

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
71,2018-03-01,2018-05-31,90,zip code,2,40833,f,Zip Code: 98121,NaN,Washington,...,-0.108458,NaN,NaN,NaN,0.777778,-0.011696,-0.116959,"Seattle, WA",42644,2024-03-10 14:36:40
1907,2014-05-01,2014-07-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.004266,NaN,NaN,NaN,0.712121,-0.129984,0.109971,"Seattle, WA",42644,2024-03-10 14:36:40
3522,2014-09-01,2014-11-30,90,zip code,2,40865,f,Zip Code: 98175,NaN,Washington,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4375,2013-11-01,2014-01-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4781,2018-09-01,2018-11-30,90,zip code,2,40834,f,Zip Code: 98122,NaN,Washington,...,-0.411765,NaN,NaN,NaN,0.333333,0.183333,-0.066667,"Seattle, WA",42644,2024-03-10 14:36:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,zip code,2,40821,f,Zip Code: 98108,NaN,Washington,...,-0.083333,NaN,NaN,NaN,0.800000,0.400000,0.466667,"Seattle, WA",42644,2024-03-10 14:36:40
7842141,2016-03-01,2016-05-31,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.233333,NaN,NaN,NaN,0.500000,-0.500000,-0.500000,"Seattle, WA",42644,2024-03-10 14:36:40
7842261,2022-03-01,2022-05-31,90,zip code,2,40825,f,Zip Code: 98112,NaN,Washington,...,0.293651,NaN,NaN,NaN,0.888889,0.111111,-0.111111,"Seattle, WA",42644,2024-03-10 14:36:40
7842290,2015-07-01,2015-09-30,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.666667,NaN,NaN,NaN,0.500000,-0.166667,NaN,"Seattle, WA",42644,2024-03-10 14:36:40


In [ ]:
# The dataset has the following columns:

# period_begin
# period_end
# period_duration
# region_type
# region_type_id
# table_id
# is_seasonally_adjusted. (indicates if prices are seasonally adjusted; f represents False)
# region
# city
# state
# state_code
# property_type
# property_type_id
# median_sale_price
# median_sale_price_mom (median sale price changes month over month)
# median_sale_price_yoy (median sale price changes year over year)
# median_list_price
# median_list_price_mom (median list price changes month over month)
# median_list_price_yoy (median list price changes year over year)
# median_ppsf (median sale price per square foot)
# median_ppsf_mom (median sale price per square foot changes month over month)
# median_ppsf_yoy (median sale price per square foot changes year over year)
# median_list_ppsf (median list price per square foot)
# median_list_ppsf_mom (median list price per square foot changes month over month)
# median_list_ppsf_yoy. (median list price per square foot changes year over year)
# homes_sold (number of homes sold)
# homes_sold_mom (number of homes sold month over month)
# homes_sold_yoy (number of homes sold year over year)
# pending_sales
# pending_sales_mom
# pending_sales_yoy
# new_listings
# new_listings_mom
# new_listings_yoy
# inventory
# inventory_mom
# inventory_yoy
# months_of_supply
# months_of_supply_mom
# months_of_supply_yoy
# median_dom (median days on market until property is sold)
# median_dom_mom (median days on market changes month over month)
# median_dom_yoy (median days on market changes year over year)
# avg_sale_to_list (average sale price to list price ratio)
# avg_sale_to_list_mom (average sale price to list price ratio changes month over month)
# avg_sale_to_list_yoy (average sale price to list price ratio changes year over year)
# sold_above_list
# sold_above_list_mom
# sold_above_list_yoy
# price_drops
# price_drops_mom
# price_drops_yoy
# off_market_in_two_weeks (number of properties that will be taken off the market within 2 weeks)
# off_market_in_two_weeks_mom (changes in number of properties that will be taken off the market within 2 weeks, month over month)
# off_market_in_two_weeks_yoy (changes in number of properties that will be taken off the market within 2 weeks, year over year)
# parent_metro_region
# parent_metro_region_metro_code
# last_updated

In [43]:
#Begin narrowing down the colunms. all of these could be useful potentially but its a lot to track
metrodata2 = metrodata[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
# metrodata2.reset_index().sort_values(by="period_begin", ascending = True)
metrodata2.reset_index().head()
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
metrodata2

,period_begin,period_end,period_duration,table_id,region,city,state,state_code,property_type,property_type_id,...,inventory_yoy,months_of_supply,months_of_supply_yoy,avg_sale_to_list,avg_sale_to_list_yoy,sold_above_list,sold_above_list_yoy,price_drops,price_drops_yoy,parent_metro_region
71,2018-03-01,2018-05-31,90,40833,Zip Code: 98121,NaN,Washington,WA,All Residential,-1,...,1.476190,NaN,NaN,1.037550,-0.010136,0.520408,-0.108458,NaN,NaN,"Seattle, WA"
1907,2014-05-01,2014-07-31,90,40816,Zip Code: 98103,NaN,Washington,WA,All Residential,-1,...,0.034884,NaN,NaN,1.030420,0.000531,0.545082,0.004266,NaN,NaN,"Seattle, WA"
3522,2014-09-01,2014-11-30,90,40865,Zip Code: 98175,NaN,Washington,WA,Townhouse,13,...,NaN,NaN,NaN,0.986301,NaN,0.000000,NaN,NaN,NaN,"Seattle, WA"
4375,2013-11-01,2014-01-31,90,40816,Zip Code: 98103,NaN,Washington,WA,Multi-Family (2-4 Unit),4,...,0.333333,NaN,NaN,0.969471,0.002038,0.000000,0.000000,NaN,NaN,"Seattle, WA"
4781,2018-09-01,2018-11-30,90,40834,Zip Code: 98122,NaN,Washington,WA,Townhouse,13,...,5.000000,NaN,NaN,0.989249,-0.036230,0.088235,-0.411765,NaN,NaN,"Seattle, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,40821,Zip Code: 98108,NaN,Washington,WA,Townhouse,13,...,-0.333333,NaN,NaN,0.990874,0.008804,0.000000,-0.083333,NaN,NaN,"Seattle, WA"
7842141,2016-03-01,2016-05-31,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,1.500000,NaN,NaN,1.007034,0.023603,0.400000,0.233333,NaN,NaN,"Seattle, WA"
7842261,2022-03-01,2022-05-31,90,40825,Zip Code: 98112,NaN,Washington,WA,Condo/Co-op,3,...,-0.636364,NaN,NaN,1.033408,0.025816,0.571429,0.293651,NaN,NaN,"Seattle, WA"
7842290,2015-07-01,2015-09-30,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,-0.500000,NaN,NaN,1.026968,0.052054,0.666667,0.666667,NaN,NaN,"Seattle, WA"


In [39]:
# do we want to group by property type? (condo, townhouse, single-family?)
groupedbypropertytype = metrodata2.groupby("property_type")
groupedbypropertytype

In [9]:
# metrodata = metrodata[
#     'period_begin',
#     'period_end',
#     'period_duration',
#     'region_type',
#     'region_type_id',
#     'table_id',
#     'is_seasonally_adjusted',
#     'region',
#     'city',
#     'state',
#     'state_code',
#     'property_type',
#     'property_type_id',
#     'median_sale_price',
#     'median_sale_price_mom',
#     'median_sale_price_yoy',
#     'median_list_price',
#     'median_list_price_mom',
#     'median_list_price_yoy',
#     'median_ppsf',
#     'median_ppsf_mom',
#     'median_ppsf_yoy',
#     'median_list_ppsf',
#     'median_list_ppsf_mom',
#     'median_list_ppsf_yoy',
#     'homes_sold',
#     'homes_sold_mom',
#     'homes_sold_yoy',
#     'pending_sales',
#     'pending_sales_mom',
#     'pending_sales_yoy',
#     'new_listings',
#     'new_listings_mom',
#     'new_listings_yoy',
#     'inventory',
#     'inventory_mom',
#     'inventory_yoy',
#     'months_of_supply',
#     'months_of_supply_mom',
#     'months_of_supply_yoy',
#     'median_dom',
#     'median_dom_mom',
#     'median_dom_yoy',
#     'avg_sale_to_list',
#     'avg_sale_to_list_mom',
#     'avg_sale_to_list_yoy',
#     'sold_above_list',
#     'sold_above_list_mom',
#     'sold_above_list_yoy',
#     'price_drops',
#     'price_drops_mom',
#     'price_drops_yoy',
#     'off_market_in_two_weeks',
#     'off_market_in_two_weeks_mom',
#     'off_market_in_two_weeks_yoy',
#     'parent_metro_region',
#     'parent_metro_region_metro_code',
#     'last_updated']

In [73]:
# Save the data for the seattle metro area with only potentially useful series headers
metrodata2.to_csv("../data_sources/redfin_by_zipcode.csv", index=False)

# Section Sale Prices

In [287]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [288]:
# metrodata2 = metrodata2[[
#     'period_begin',
#     'period_end',
#     'table_id',
#     'region',
#     'city',
#     'property_type',
#     'property_type_id',
#     'median_sale_price',
#     'median_sale_price_yoy',
#     'median_list_price',
#     'median_list_price_yoy',
#     'median_ppsf',
#     'median_ppsf_yoy',
#     'median_list_ppsf',
#     'median_list_ppsf_yoy',
#     'homes_sold',
#     'homes_sold_yoy',
#     'pending_sales',
#     'pending_sales_yoy',
#     'new_listings',
#     'new_listings_yoy',
#     'inventory',
#     'inventory_mom',
#     'inventory_yoy',
#     'avg_sale_to_list',
#     'avg_sale_to_list_yoy',
#     'sold_above_list',
#     'sold_above_list_yoy',
#     'parent_metro_region' 
# ]]


In [289]:
#Here are what might be the best data points in the dataframe
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [290]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,Zip Code: 98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,Zip Code: 98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,Zip Code: 98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,Zip Code: 98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [291]:
# Sort by period_end and group by unique zipcode
metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby('region')
groupedmetrodata2_sorted

In [292]:
# Cool Function to chaneg the date time (dt) of period_end to the quarter
quarters = metrodata2['period_end'].dt.to_period('Q').unique().strftime('%Y-Q%q')

# Empty dictionary to store mean_median_sale_price for each quarter and each group(zipcode)

zipcode_and_mean_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}

#For-loop to go through the grouped data to find the mean_median_sale_price for every quarter
for zipcode, group in groupedmetrodata2_sorted:
    for quarter in quarters:
        mean_median_sale_price = group[group['period_end'].dt.to_period('Q') == quarter]['median_sale_price'].mean()
        zipcode_and_mean_dict[zipcode].append(mean_median_sale_price)

#Put this data from the loop into a Dataframe with the dictionary keys and datetime
mean_sales_df = pd.DataFrame(zipcode_and_mean_dict, index=quarters)

# Transpose Dataframe. The df.T will switch columns and rows
mean_sales_df = mean_sales_df.T

In [293]:
#Drop the decimal places and make it an int instead of float
mean_sales_df = mean_sales_df.dropna().round(0).astype(int)
mean_sales_df = mean_sales_df.sort_index(axis=1)
mean_sales_df

,2012-Q1,2012-Q2,2012-Q3,2012-Q4,2013-Q1,2013-Q2,2013-Q3,2013-Q4,2014-Q1,2014-Q2,...,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1
Zip Code: 98101,632000,480167,503150,499833,578000,531667,421683,473167,697667,528167,...,863425,942889,745000,631167,646454,538833,658333,652128,600278,850000
Zip Code: 98102,478400,469706,472678,510033,481990,563115,574748,713090,588084,610917,...,1033128,1118357,1077755,1052790,991762,971769,1000496,1043535,1071667,829414
Zip Code: 98103,396300,440550,407155,394756,408807,449780,443280,459820,472632,460561,...,788623,847252,875080,868017,905467,851765,938605,942392,912098,880175
Zip Code: 98104,324833,302212,206200,246000,301667,354000,311833,491003,770344,533264,...,562483,393975,420808,419158,376642,415892,413967,408892,381217,453400
Zip Code: 98105,458600,508259,500059,483693,441060,504517,541387,491943,533933,538900,...,899382,1026050,1227836,1190863,1051495,975626,1057597,985137,931402,1015833
Zip Code: 98106,205500,179825,201010,198691,228532,232750,258152,268311,237871,264536,...,648380,640167,649590,694993,611561,615547,639659,617168,611327,660428
Zip Code: 98107,374950,385832,416759,426280,411113,422800,460415,465433,484125,496783,...,836380,933493,981430,888598,763613,851433,880923,913830,1061333,1088500
Zip Code: 98108,206250,238739,264486,265780,280402,288827,312840,307200,320343,312250,...,605393,717614,696830,631830,700550,602120,618758,681423,709951,727073
Zip Code: 98109,457250,492462,474910,509993,540981,512631,511593,499143,527750,592986,...,1080148,941621,1124170,1074154,942083,941055,1003354,1210531,1111392,1043325
Zip Code: 98112,599811,578349,545300,682722,691416,670786,671330,628355,623583,676000,...,1130128,1254533,1338163,1288017,1183615,1131194,1100720,1265363,1202545,1111584


In [294]:
mean_sales_df.index = mean_sales_df.index.str.extract('(\d{5})')[0]

In [295]:
mean_sales_df

,2012-Q1,2012-Q2,2012-Q3,2012-Q4,2013-Q1,2013-Q2,2013-Q3,2013-Q4,2014-Q1,2014-Q2,...,2021-Q4,2022-Q1,2022-Q2,2022-Q3,2022-Q4,2023-Q1,2023-Q2,2023-Q3,2023-Q4,2024-Q1
0,,,,,,,,,,,,,,,,,,,,,
98101,632000,480167,503150,499833,578000,531667,421683,473167,697667,528167,...,863425,942889,745000,631167,646454,538833,658333,652128,600278,850000
98102,478400,469706,472678,510033,481990,563115,574748,713090,588084,610917,...,1033128,1118357,1077755,1052790,991762,971769,1000496,1043535,1071667,829414
98103,396300,440550,407155,394756,408807,449780,443280,459820,472632,460561,...,788623,847252,875080,868017,905467,851765,938605,942392,912098,880175
98104,324833,302212,206200,246000,301667,354000,311833,491003,770344,533264,...,562483,393975,420808,419158,376642,415892,413967,408892,381217,453400
98105,458600,508259,500059,483693,441060,504517,541387,491943,533933,538900,...,899382,1026050,1227836,1190863,1051495,975626,1057597,985137,931402,1015833
98106,205500,179825,201010,198691,228532,232750,258152,268311,237871,264536,...,648380,640167,649590,694993,611561,615547,639659,617168,611327,660428
98107,374950,385832,416759,426280,411113,422800,460415,465433,484125,496783,...,836380,933493,981430,888598,763613,851433,880923,913830,1061333,1088500
98108,206250,238739,264486,265780,280402,288827,312840,307200,320343,312250,...,605393,717614,696830,631830,700550,602120,618758,681423,709951,727073
98109,457250,492462,474910,509993,540981,512631,511593,499143,527750,592986,...,1080148,941621,1124170,1074154,942083,941055,1003354,1210531,1111392,1043325


In [ ]:
# Save the data for the Average_Median_Sale_Price of homes in seattle zipcodes by quarter
mean_sales_df.to_csv("../data_sources/average_median_sale_price_by_quarter.csv", index=True)

In [296]:
collistt = list(mean_sales_df.columns)
collistt

['2012-Q1',
 '2012-Q2',
 '2012-Q3',
 '2012-Q4',
 '2013-Q1',
 '2013-Q2',
 '2013-Q3',
 '2013-Q4',
 '2014-Q1',
 '2014-Q2',
 '2014-Q3',
 '2014-Q4',
 '2015-Q1',
 '2015-Q2',
 '2015-Q3',
 '2015-Q4',
 '2016-Q1',
 '2016-Q2',
 '2016-Q3',
 '2016-Q4',
 '2017-Q1',
 '2017-Q2',
 '2017-Q3',
 '2017-Q4',
 '2018-Q1',
 '2018-Q2',
 '2018-Q3',
 '2018-Q4',
 '2019-Q1',
 '2019-Q2',
 '2019-Q3',
 '2019-Q4',
 '2020-Q1',
 '2020-Q2',
 '2020-Q3',
 '2020-Q4',
 '2021-Q1',
 '2021-Q2',
 '2021-Q3',
 '2021-Q4',
 '2022-Q1',
 '2022-Q2',
 '2022-Q3',
 '2022-Q4',
 '2023-Q1',
 '2023-Q2',
 '2023-Q3',
 '2023-Q4',
 '2024-Q1']

In [297]:
mean_sales_df = mean_sales_df[['2017-Q1',
 '2017-Q2',
 '2017-Q3',
 '2017-Q4',
 '2018-Q1',
 '2018-Q2',
 '2018-Q3',
 '2018-Q4',
 '2019-Q1',
 '2019-Q2',
 '2019-Q3',
 '2019-Q4',
 '2020-Q1',
 '2020-Q2',
 '2020-Q3',
 '2020-Q4',
 '2021-Q1',
 '2021-Q2',
 '2021-Q3',
 '2021-Q4']]

In [298]:
mean_sales_df.head(1)

,2017-Q1,2017-Q2,2017-Q3,2017-Q4,2018-Q1,2018-Q2,2018-Q3,2018-Q4,2019-Q1,2019-Q2,2019-Q3,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4
0,,,,,,,,,,,,,,,,,,,,
98101,629967,614167,639467,691667,798633,784300,753333,778850,712483,639000,705417,656067,587417,641642,812367,682636,675500,633417,665332,863425


In [301]:
# Save the data for the Average_Median_Sale_Price of homes in seattle zipcodes by quarter
mean_sales_df.to_csv("../data_frames/average_median_sale_price_by_quarter2017-2021.csv", index=True)

# Section Sale Price filtering for average

In [ ]:
# average_median_sale_price_by_quarter.csv

In [305]:
filepath_salepriceavg = "../data_frames/average_median_sale_price_by_quarter2017-2021.csv"
saleprice_avg = pd.read_csv(filepath_salepriceavg, index_col=0)
saleprice_avg.head()

,2017-Q1,2017-Q2,2017-Q3,2017-Q4,2018-Q1,2018-Q2,2018-Q3,2018-Q4,2019-Q1,2019-Q2,2019-Q3,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4
0,,,,,,,,,,,,,,,,,,,,
98101,629967,614167,639467,691667,798633,784300,753333,778850,712483,639000,705417,656067,587417,641642,812367,682636,675500,633417,665332,863425
98102,847094,856167,824820,1011056,1061160,1123953,1099862,1154782,994948,904183,912748,987686,976643,1033230,1050193,1035250,931803,991560,1445086,1033128
98103,670310,689895,736618,736968,784596,832872,826282,758228,757417,768683,764391,730155,756017,748912,779648,739083,763935,835932,840817,788623
98104,322500,346667,421884,541667,556133,664800,701333,404000,579833,663975,599667,529000,578333,620650,625000,518750,562333,620617,582379,562483
98105,697592,769673,843404,824700,830869,928014,910872,889047,833879,818875,945965,833618,795855,881382,934495,903188,882166,912276,930376,899382


In [309]:
saleprice_avg.index.name = "Zipcode"

In [310]:
saleprice_avg.head(1)

,2017-Q1,2017-Q2,2017-Q3,2017-Q4,2018-Q1,2018-Q2,2018-Q3,2018-Q4,2019-Q1,2019-Q2,2019-Q3,2019-Q4,2020-Q1,2020-Q2,2020-Q3,2020-Q4,2021-Q1,2021-Q2,2021-Q3,2021-Q4
Zipcode,,,,,,,,,,,,,,,,,,,,
98101,629967,614167,639467,691667,798633,784300,753333,778850,712483,639000,705417,656067,587417,641642,812367,682636,675500,633417,665332,863425


In [269]:
# zipcodes = list(saleprice_avg['Zipcode'])

In [311]:
saleprice_averages_by_quarter = saleprice_avg.mean()
saleprice_averages_by_quarter.to_frame('Avg Price all Zipcodes')


,Avg Price all Zipcodes
2017-Q1,602734.384615
2017-Q2,639774.884615
2017-Q3,660199.692308
2017-Q4,675987.384615
2018-Q1,718276.000000
2018-Q2,760614.192308
2018-Q3,731133.807692
2018-Q4,702407.923077
2019-Q1,683218.076923
2019-Q2,702495.307692


In [312]:
# Save the data and keep the index for quarters
saleprice_averages_by_quarter.to_csv("../data_frames/saleprice_quarterly_AVG.csv", index=True)

In [ ]:
# Average Cost for Each ZIP Code

In [314]:
sale_zipcode_averages = saleprice_avg.mean(axis=1)
sale_zipcode_averages = sale_zipcode_averages.to_frame(name='Avg price per Zipcode 2017-2021')
sale_zipcode_averages

,Avg price per Zipcode 2017-2021
Zipcode,
98101,698254.35
98102,1013767.60
98103,765469.10
98104,550100.20
98105,863281.40
98106,533400.15
98107,795666.45
98108,588026.40
98109,919703.55


In [315]:
# Save the data and keep the index for zipcodes
sale_zipcode_averages.to_csv("../data_frames/zipcode_sale_AVG.csv", index=True)

In [318]:
sale_top_zipcodes = sale_zipcode_averages.nlargest(5, 'Avg price per Zipcode 2017-2021')

sale_bottom_zipcodes = sale_zipcode_averages.nsmallest(5, 'Avg price per Zipcode 2017-2021')

In [319]:
sale_top_zipcodes

,Avg price per Zipcode 2017-2021
Zipcode,
98112,1045929.95
98102,1013767.60
98109,919703.55
98119,914427.65
98105,863281.40


In [320]:
sale_bottom_zipcodes

,Avg price per Zipcode 2017-2021
Zipcode,
98178,422227.65
98146,436521.75
98106,533400.15
98104,550100.20
98133,572678.90


# Section Property Type and Inventory

In [3]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [4]:
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [5]:
# change the region series to zipcode and format the string
metrodata2['Zipcode'] = metrodata2['region'].str.extract('(\\d{5})')
metrodata2 = metrodata2.drop(columns=['region'])

In [6]:
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'Zipcode',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [7]:
metrodata2 

,period_begin,period_end,Zipcode,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [8]:
property_types = metrodata2['property_type'].unique()
print(property_types)

['All Residential' 'Townhouse' 'Multi-Family (2-4 Unit)' 'Condo/Co-op'
 'Single Family Residential']


In [9]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,Zipcode,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [10]:
#NEW PATH
metrodata2_sorted =  metrodata2.sort_values(by='Zipcode')

In [11]:
#first for property_type='All Residential'
#then for property_type!='All Residential'

In [12]:
all_residential_data = metrodata2_sorted[metrodata2_sorted['property_type'] == 'All Residential']
all_residential_data

,period_begin,period_end,Zipcode,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
859,2014-01-01,2014-03-31,98101,All Residential,733000.0,544500.0,581.318681,43.0,52.0,22.0,0.985031,0.139535
1860,2012-10-01,2012-12-31,98101,All Residential,499750.0,547000.0,494.565089,64.0,52.0,45.0,0.977436,0.125000
1213,2015-07-01,2015-09-30,98101,All Residential,675200.0,584995.0,708.075773,115.0,72.0,61.0,1.002584,0.130435
265,2018-04-01,2018-06-30,98101,All Residential,780500.0,795000.0,824.403941,72.0,88.0,61.0,1.000114,0.291667
263,2013-03-01,2013-05-31,98101,All Residential,540000.0,490000.0,567.226891,61.0,58.0,54.0,0.981864,0.081967
...,...,...,...,...,...,...,...,...,...,...,...,...
13534,2012-04-01,2012-06-30,98199,All Residential,494000.0,549500.0,254.237288,113.0,131.0,82.0,0.991403,0.265487
13482,2017-06-01,2017-08-31,98199,All Residential,845000.0,800000.0,441.866792,166.0,173.0,30.0,1.034883,0.439759
10684,2020-10-01,2020-12-31,98199,All Residential,1009500.0,935000.0,492.063657,122.0,109.0,37.0,0.998818,0.295082
13446,2015-09-01,2015-11-30,98199,All Residential,645000.0,702450.0,346.428571,101.0,112.0,30.0,1.025881,0.495050


In [293]:
all_residential_data.set_index('period_end', inplace=True)

In [258]:
#This should rename the perdiod end and give it a value for month
monthly_inventory = all_residential_data.groupby('Zipcode').resample('M')['inventory'].mean()
monthly_inventory = monthly_inventory.reset_index()
monthly_inventory

,Zipcode,period_end,inventory
0,98101,2012-03-31,54.0
1,98101,2012-04-30,60.0
2,98101,2012-05-31,53.0
3,98101,2012-06-30,60.0
4,98101,2012-07-31,58.0
...,...,...,...
4327,98199,2023-10-31,50.0
4328,98199,2023-11-30,41.0
4329,98199,2023-12-31,21.0
4330,98199,2024-01-31,17.0


In [260]:
# Filter for dates of interest (2018-2022)
monthly_inventory_filtered = monthly_inventory[(monthly_inventory['period_end'] >= '2018-01-01') & (monthly_inventory['period_end'] <= '2022-12-31')]
monthly_inventory_filtered

,Zipcode,period_end,inventory
70,98101,2018-01-31,18.0
71,98101,2018-02-28,44.0
72,98101,2018-03-31,46.0
73,98101,2018-04-30,53.0
74,98101,2018-05-31,61.0
...,...,...,...
4313,98199,2022-08-31,59.0
4314,98199,2022-09-30,70.0
4315,98199,2022-10-31,61.0
4316,98199,2022-11-30,50.0


In [261]:
#Save monthly_inventory for all types of sales for the time frame based on zipcode - ready to graph and analyze
monthly_inventory_filtered.to_csv("../data_frames/monthly_housingsale_available_inventory.csv", index=True)

In [ ]:
#NOW FOR QUARTERS


In [263]:
quarterly_inventory = all_residential_data.groupby('Zipcode').resample('Q')['inventory'].mean()
quarterly_inventory = quarterly_inventory.reset_index()
quarterly_inventory

,Zipcode,period_end,inventory
0,98101,2012-03-31,54.000000
1,98101,2012-06-30,57.666667
2,98101,2012-09-30,57.000000
3,98101,2012-12-31,45.333333
4,98101,2013-03-31,38.333333
...,...,...,...
1469,98199,2023-03-31,26.666667
1470,98199,2023-06-30,41.000000
1471,98199,2023-09-30,44.000000
1472,98199,2023-12-31,37.333333


In [264]:
# Filter for dates of interest (2018-2022)
quarterly_inventory_filtered = quarterly_inventory[(quarterly_inventory['period_end'] >= '2018-01-01') & (quarterly_inventory['period_end'] <= '2022-12-31')]
quarterly_inventory_filtered

,Zipcode,period_end,inventory
24,98101,2018-03-31,36.000000
25,98101,2018-06-30,58.333333
26,98101,2018-09-30,46.333333
27,98101,2018-12-31,49.333333
28,98101,2019-03-31,48.666667
...,...,...,...
1464,98199,2021-12-31,16.000000
1465,98199,2022-03-31,14.333333
1466,98199,2022-06-30,41.333333
1467,98199,2022-09-30,64.000000


In [265]:
#Save quarterly_inventory for all types of sales for the time frame based on zipcode - ready to graph and analyze
quarterly_inventory_filtered.to_csv("../data_frames/quarterly_housingsale_available_inventory.csv", index=True)

In [266]:
#NOW FOR YEAR
yearly_inventory = all_residential_data.groupby('Zipcode').resample('Y')['inventory'].mean()
yearly_inventory = yearly_inventory.reset_index()
yearly_inventory

,Zipcode,period_end,inventory
0,98101,2012-12-31,53.400000
1,98101,2013-12-31,38.500000
2,98101,2014-12-31,26.666667
3,98101,2015-12-31,43.833333
4,98101,2016-12-31,33.916667
...,...,...,...
386,98199,2020-12-31,48.083333
387,98199,2021-12-31,33.250000
388,98199,2022-12-31,41.916667
389,98199,2023-12-31,37.250000


In [267]:
# Filter for dates of interest (2018-2022)
yearly_inventory_filtered = yearly_inventory[(yearly_inventory['period_end'] >= '2018-01-01') & (yearly_inventory['period_end'] <= '2022-12-31')]
yearly_inventory_filtered

,Zipcode,period_end,inventory
6,98101,2018-12-31,47.500000
7,98101,2019-12-31,56.250000
8,98101,2020-12-31,82.000000
9,98101,2021-12-31,59.833333
10,98101,2022-12-31,57.250000
...,...,...,...
384,98199,2018-12-31,47.583333
385,98199,2019-12-31,59.916667
386,98199,2020-12-31,48.083333
387,98199,2021-12-31,33.250000


In [269]:
#Save yearly_inventory for all types of sales for the time frame based on zipcode - ready to graph and analyze
yearly_inventory_filtered.to_csv("../data_frames/yearly_housingsale_available_inventory.csv", index=True)

In [ ]:
#FROM NEW PATH ABOVE TO GRAB ALL YEARS


In [13]:
all_residential_data = metrodata2_sorted[metrodata2_sorted['property_type'] == 'All Residential']
all_residential_data

,period_begin,period_end,Zipcode,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
859,2014-01-01,2014-03-31,98101,All Residential,733000.0,544500.0,581.318681,43.0,52.0,22.0,0.985031,0.139535
1860,2012-10-01,2012-12-31,98101,All Residential,499750.0,547000.0,494.565089,64.0,52.0,45.0,0.977436,0.125000
1213,2015-07-01,2015-09-30,98101,All Residential,675200.0,584995.0,708.075773,115.0,72.0,61.0,1.002584,0.130435
265,2018-04-01,2018-06-30,98101,All Residential,780500.0,795000.0,824.403941,72.0,88.0,61.0,1.000114,0.291667
263,2013-03-01,2013-05-31,98101,All Residential,540000.0,490000.0,567.226891,61.0,58.0,54.0,0.981864,0.081967
...,...,...,...,...,...,...,...,...,...,...,...,...
13534,2012-04-01,2012-06-30,98199,All Residential,494000.0,549500.0,254.237288,113.0,131.0,82.0,0.991403,0.265487
13482,2017-06-01,2017-08-31,98199,All Residential,845000.0,800000.0,441.866792,166.0,173.0,30.0,1.034883,0.439759
10684,2020-10-01,2020-12-31,98199,All Residential,1009500.0,935000.0,492.063657,122.0,109.0,37.0,0.998818,0.295082
13446,2015-09-01,2015-11-30,98199,All Residential,645000.0,702450.0,346.428571,101.0,112.0,30.0,1.025881,0.495050


In [14]:
all_residential_data.set_index('period_end', inplace=True)

In [15]:
ALLyearly_inventory = all_residential_data.groupby('Zipcode').resample('Y')['inventory'].mean()
ALLyearly_inventory = ALLyearly_inventory.reset_index()
ALLyearly_inventory

,Zipcode,period_end,inventory
0,98101,2012-12-31,53.400000
1,98101,2013-12-31,38.500000
2,98101,2014-12-31,26.666667
3,98101,2015-12-31,43.833333
4,98101,2016-12-31,33.916667
...,...,...,...
386,98199,2020-12-31,48.083333
387,98199,2021-12-31,33.250000
388,98199,2022-12-31,41.916667
389,98199,2023-12-31,37.250000


In [1]:
# yearly_inventory

In [16]:
#Save yearly_inventory for all types of sales for the time frame based on zipcode - ready to graph and analyze
ALLyearly_inventory.to_csv("../data_frames/ALLYEARS_housingsale_available_inventory.csv", index=True)

In [ ]:
#FROM NEW PATH TO GO BY GROUP

In [262]:
typed_residential_data = metrodata2_sorted[metrodata2_sorted['property_type'] != 'All Residential']
typed_residential_data

,period_begin,period_end,Zipcode,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
651,2023-05-01,2023-07-31,98101,Condo/Co-op,711386.0,730750.0,846.443317,56.0,92.0,63.0,0.977239,0.160714
281,2017-09-01,2017-11-30,98101,Condo/Co-op,675000.0,685000.0,738.993711,47.0,59.0,29.0,1.033780,0.531915
1863,2019-10-01,2019-12-31,98101,Condo/Co-op,620000.0,645000.0,771.130381,44.0,36.0,37.0,0.976960,0.022727
7951,2017-08-01,2017-10-31,98101,Condo/Co-op,720000.0,810000.0,769.230769,43.0,67.0,32.0,1.046508,0.604651
11195,2022-02-01,2022-04-30,98101,Condo/Co-op,750000.0,719900.0,841.544880,81.0,115.0,55.0,0.999535,0.271605
...,...,...,...,...,...,...,...,...,...,...,...,...
13452,2013-03-01,2013-05-31,98199,Single Family Residential,662500.0,675000.0,281.984974,92.0,113.0,40.0,1.020626,0.532609
2120,2020-03-01,2020-05-31,98199,Single Family Residential,1205000.0,1195000.0,472.000000,71.0,92.0,29.0,1.013841,0.408451
5263,2016-07-01,2016-09-30,98199,Multi-Family (2-4 Unit),1130000.0,599950.0,290.291262,5.0,3.0,1.0,1.001265,0.400000
16081,2020-08-01,2020-10-31,98199,Townhouse,730000.0,744950.0,507.845745,23.0,28.0,14.0,0.999925,0.217391


In [242]:
# Sort by period_end and group by unique zipcode
property_type_grouped_data = metrodata2.groupby(['Zipcode', 'property_type'])
property_type_grouped_data

In [ ]:
#

In [136]:
# Sort by period_end and group by unique zipcode
# metrodata2['inventory'] = metrodata2['inventory'].fillna(0)
# metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby('region')
groupedmetrodata2_sorted.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
16556,2012-01-01,2012-03-31,Zip Code: 98105,All Residential,556000.0,539950.0,270.175439,47.0,75.0,41.0,0.975642,0.148936
12764,2012-01-01,2012-03-31,Zip Code: 98105,Single Family Residential,662500.0,599500.0,271.429382,36.0,56.0,21.0,0.976863,0.194444
5438,2012-01-01,2012-03-31,Zip Code: 98116,Condo/Co-op,252250.0,235000.0,250.265055,18.0,27.0,23.0,0.962819,0.055556
5427,2012-01-01,2012-03-31,Zip Code: 98144,Condo/Co-op,185750.0,244900.0,221.331884,6.0,24.0,33.0,0.994658,0.166667
16913,2012-01-01,2012-03-31,Zip Code: 98199,Single Family Residential,530000.0,594500.0,250.000000,45.0,68.0,58.0,0.983099,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
11881,2017-01-01,2017-03-31,Zip Code: 98185,Single Family Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
8325,2017-01-01,2017-03-31,Zip Code: 98185,All Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
8112,2017-02-01,2017-04-30,Zip Code: 98185,All Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000
3219,2017-02-01,2017-04-30,Zip Code: 98185,Single Family Residential,260000.0,NaN,317.073171,1.0,NaN,0.0,0.945455,0.000000


In [144]:
# Cool Function to chaneg the date time (dt) of period_end to the month
months = metrodata2['period_end'].dt.to_period('M').unique()

# Empty dictionary to store inventory_available_houses for each month and each group(zipcode)

zipcode_and_avail_houses_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}

#For-loop to go through the grouped data to find the zipcode_and_#avail_houses for every month
for zipcode, group in groupedmetrodata2_sorted:
    for month in months:
        available_houses = group[group['period_end'].dt.to_period('M') == month]['inventory'].sum()
        zipcode_and_avail_houses_dict[zipcode].append(available_houses)

#Put this data from the loop into a Dataframe with the available_houses for each zipcode by datetime
total_house_inventory = pd.DataFrame(zipcode_and_avail_houses_dict, index=months)

# Transpose Dataframe. The df.T will switch columns and rows
total_house_inventory = total_house_inventory.T
total_house_inventory = total_house_inventory.sort_index(axis=1)

In [146]:


total_house_inventory.head()

,2012-03,2012-04,2012-05,2012-06,2012-07,2012-08,2012-09,2012-10,2012-11,2012-12,...,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02
Zip Code: 98101,107.0,119.0,105.0,119.0,116.0,106.0,120.0,96.0,86.0,90.0,...,118.0,140.0,126.0,134.0,150.0,142.0,118.0,110.0,106.0,118.0
Zip Code: 98102,118.0,138.0,124.0,148.0,132.0,118.0,122.0,94.0,102.0,88.0,...,108.0,109.0,103.0,126.0,145.0,124.0,94.0,64.0,73.0,88.0
Zip Code: 98103,178.0,188.0,192.0,190.0,208.0,200.0,226.0,162.0,130.0,94.0,...,108.0,136.0,122.0,146.0,184.0,174.0,122.0,72.0,80.0,94.0
Zip Code: 98104,36.0,34.0,40.0,47.0,35.0,33.0,33.0,33.0,27.0,25.0,...,64.0,66.0,76.0,68.0,76.0,72.0,70.0,52.0,56.0,54.0
Zip Code: 98105,82.0,86.0,102.0,110.0,130.0,110.0,98.0,94.0,84.0,66.0,...,76.0,78.0,92.0,88.0,108.0,84.0,70.0,53.0,75.0,70.0


In [147]:
# Save the data for the total_housing_availability of homes in seattle zipcodes by month
total_house_inventory.to_csv("../data_frames/total_housing_availability_by_month.csv", index=True)

# Section Housing Availability

In [131]:
filepath_metrodata2 = "../data_sources/redfin_by_zipcode.csv"
metrodata2 = pd.read_csv(filepath_metrodata2)
metrodata2.head()

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,NaN,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235


In [152]:
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'region',
    'property_type',
    'median_sale_price',
    'median_list_price',
    'median_ppsf',
    'homes_sold',
    'new_listings',
    'inventory',
    'avg_sale_to_list',
    'sold_above_list'
]]

In [153]:
property_types = metrodata2['property_type'].unique()
print(property_types)

['All Residential' 'Townhouse' 'Multi-Family (2-4 Unit)' 'Condo/Co-op'
 'Single Family Residential']


In [154]:
#Change the period_end and period_start series to date time from string
metrodata2['period_begin'] = pd.to_datetime(metrodata2['period_begin'])
metrodata2['period_end'] = pd.to_datetime(metrodata2['period_end'])
# metrodata2.sort_values(by = 'period_end', ascending=True)
metrodata2

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
0,2018-03-01,2018-05-31,Zip Code: 98121,All Residential,667500.0,625000.0,860.121526,98.0,145.0,52.0,1.037550,0.520408
1,2014-05-01,2014-07-31,Zip Code: 98103,All Residential,539000.0,535000.0,318.695652,244.0,283.0,89.0,1.030420,0.545082
2,2014-09-01,2014-11-30,Zip Code: 98175,Townhouse,360000.0,NaN,260.680666,1.0,NaN,0.0,0.986301,0.000000
3,2013-11-01,2014-01-31,Zip Code: 98103,Multi-Family (2-4 Unit),570500.0,850000.0,244.882917,2.0,1.0,4.0,0.969471,0.000000
4,2018-09-01,2018-11-30,Zip Code: 98122,Townhouse,742500.0,749000.0,546.105168,34.0,46.0,18.0,0.989249,0.088235
...,...,...,...,...,...,...,...,...,...,...,...,...
17000,2019-12-01,2020-02-29,Zip Code: 98108,Townhouse,543750.0,541500.0,438.266981,8.0,16.0,8.0,0.990874,0.000000
17001,2016-03-01,2016-05-31,Zip Code: 98146,Condo/Co-op,178000.0,167500.0,154.782609,5.0,6.0,5.0,1.007034,0.400000
17002,2022-03-01,2022-05-31,Zip Code: 98112,Condo/Co-op,715500.0,659999.0,790.378007,21.0,25.0,4.0,1.033408,0.571429
17003,2015-07-01,2015-09-30,Zip Code: 98146,Condo/Co-op,249000.0,261500.0,271.077110,6.0,10.0,2.0,1.026968,0.666667


In [155]:
# Sort by period_end and group by unique zipcode AND type of property
metrodata2['inventory'] = metrodata2['inventory'].fillna(0)
metrodata2_sorted =  metrodata2.sort_values(by='period_end')
groupedmetrodata2_sorted = metrodata2_sorted.groupby(['region', 'property_type'])
groupedmetrodata2_sorted.head()
# df.groupby(col1)[col2] 

,period_begin,period_end,region,property_type,median_sale_price,median_list_price,median_ppsf,homes_sold,new_listings,inventory,avg_sale_to_list,sold_above_list
16556,2012-01-01,2012-03-31,Zip Code: 98105,All Residential,556000.0,539950.0,270.175439,47.0,75.0,41.0,0.975642,0.148936
12764,2012-01-01,2012-03-31,Zip Code: 98105,Single Family Residential,662500.0,599500.0,271.429382,36.0,56.0,21.0,0.976863,0.194444
5438,2012-01-01,2012-03-31,Zip Code: 98116,Condo/Co-op,252250.0,235000.0,250.265055,18.0,27.0,23.0,0.962819,0.055556
5427,2012-01-01,2012-03-31,Zip Code: 98144,Condo/Co-op,185750.0,244900.0,221.331884,6.0,24.0,33.0,0.994658,0.166667
16913,2012-01-01,2012-03-31,Zip Code: 98199,Single Family Residential,530000.0,594500.0,250.000000,45.0,68.0,58.0,0.983099,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
3932,2021-03-01,2021-05-31,Zip Code: 98185,Single Family Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
3274,2021-04-01,2021-06-30,Zip Code: 98185,Single Family Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
1292,2021-04-01,2021-06-30,Zip Code: 98185,All Residential,459000.0,459000.0,628.767123,1.0,1.0,0.0,1.000000,0.000000
5674,2021-09-01,2021-11-30,Zip Code: 98101,Townhouse,1387500.0,1345000.0,685.798157,2.0,4.0,1.0,1.030466,0.500000


In [156]:
groupedmetrodata2_sorted.fillna(0, inplace=True)

""


In [157]:
# Cool Function to chaneg the date time (dt) of period_end to the month
months = metrodata2['period_end'].dt.to_period('M').unique()

# Empty dictionary to store available houses for each property type for each month and each group(zipcode)
avail_houses_by_type_dict = {zipcode: [] for zipcode in groupedmetrodata2_sorted.groups.keys()}



#For-loop to go through the grouped data to find the mean_median_sale_price for every quarter
for (zipcode, property_type), group in groupedmetrodata2_sorted:
    avail_houses_by_type_dict[(zipcode, property_type)] = []
    for month in months:
        available_houses = group[group['period_end'].dt.to_period('M') == month]['inventory'].sum()
        avail_houses_by_type_dict[(zipcode, property_type)].append(available_houses)

        
index = pd.MultiIndex.from_tuples(avail_houses_by_type_dict.keys(), names=['zipcode', 'property_type'])


#Put this data from the loop into a Dataframe with the number of available_house for each propertytype and zipcode by month
total_house_inventory_for_property_type = pd.DataFrame(avail_houses_by_type_dict.values(), index=index, columns=months)


# Transpose Dataframe. The df.T will switch columns and rows
# total_house_inventory_for_property_type = total_house_inventory_for_property_type.T

#Sort by the multiindex
total_house_inventory_for_property_type = total_house_inventory_for_property_type.sort_index()



In [158]:
total_house_inventory_for_property_type

2018-05  2014-07  2014-11  2014-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               61.0     39.0     19.0     19.0   
                Condo/Co-op                   61.0     39.0     19.0     19.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               36.0     50.0     33.0     37.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               40.0     84.0     39.0     44.0   
                Condo/Co-op                    7.0     12.0      3.0      1.0   
                Multi-Family (2-4 Unit)        2.0      5.0      0.0      0.0   
                Single Family Residential     29.0     59.0     34.0     38.0   
                Townhouse                      2.0      8.0      2.0      3.0   

                                           2018-11  2022-03  2020-09  2012-09  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               50.0     49.0    130.0     60.0   
                Condo/Co-op                   50.0     49.0    130.0     60.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               58.0     31.0    120.0     61.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               55.0     18.0     65.0     87.0   
                Condo/Co-op                   13.0      3.0     11.0     15.0   
                Multi-Family (2-4 Unit)        1.0      0.0      3.0      2.0   
                Single Family Residential     32.0     12.0     42.0     59.0   
                Townhouse                      9.0      3.0      9.0     11.0   

                                           2014-06  2023-12  ...  2018-01  \
zipcode         property_type                                ...            
Zip Code: 98101 All Residential               31.0     55.0  ...     18.0   
                Condo/Co-op                   31.0     55.0  ...     18.0   
                Single Family Residential      0.0      0.0  ...      0.0   
                Townhouse                      0.0      0.0  ...      0.0   
Zip Code: 98102 All Residential               51.0     32.0  ...     20.0   
...                                            ...      ...  ...      ...   
Zip Code: 98199 All Residential               70.0     21.0  ...     21.0   
                Condo/Co-op                   11.0      4.0  ...      5.0   
                Multi-Family (2-4 Unit)        5.0      0.0  ...      0.0   
                Single Family Residential     49.0     12.0  ...     15.0   
                Townhouse                      5.0      5.0  ...      1.0   

                                           2015-08  2022-02  2020-11  2013-10  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               57.0     44.0     89.0     38.0   
                Condo/Co-op                   57.0     44.0     89.0     38.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               35.0     29.0    100.0     57.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               37.0     13.0     61.0     76.0   
                Condo/Co-op                    4.0      4.0     13.0     13.0   
                Multi-Family (2-4 Unit)        1.0      0.0      4.0      2.0   
                Single Family Residential 

In [203]:
# Save the data for the housing_availability_by_zipcode_property_type of homes in seattle zipcodes by month
total_house_inventory_for_property_type.to_csv("../data_frames/housing_availability_by_zip_and_property_type.csv", index=True)

In [219]:
inventorydf = total_house_inventory_for_property_type.copy()
inventorydf

2018-05  2014-07  2014-11  2014-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               61.0     39.0     19.0     19.0   
                Condo/Co-op                   61.0     39.0     19.0     19.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               36.0     50.0     33.0     37.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               40.0     84.0     39.0     44.0   
                Condo/Co-op                    7.0     12.0      3.0      1.0   
                Multi-Family (2-4 Unit)        2.0      5.0      0.0      0.0   
                Single Family Residential     29.0     59.0     34.0     38.0   
                Townhouse                      2.0      8.0      2.0      3.0   

                                           2018-11  2022-03  2020-09  2012-09  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               50.0     49.0    130.0     60.0   
                Condo/Co-op                   50.0     49.0    130.0     60.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               58.0     31.0    120.0     61.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               55.0     18.0     65.0     87.0   
                Condo/Co-op                   13.0      3.0     11.0     15.0   
                Multi-Family (2-4 Unit)        1.0      0.0      3.0      2.0   
                Single Family Residential     32.0     12.0     42.0     59.0   
                Townhouse                      9.0      3.0      9.0     11.0   

                                           2014-06  2023-12  ...  2018-01  \
zipcode         property_type                                ...            
Zip Code: 98101 All Residential               31.0     55.0  ...     18.0   
                Condo/Co-op                   31.0     55.0  ...     18.0   
                Single Family Residential      0.0      0.0  ...      0.0   
                Townhouse                      0.0      0.0  ...      0.0   
Zip Code: 98102 All Residential               51.0     32.0  ...     20.0   
...                                            ...      ...  ...      ...   
Zip Code: 98199 All Residential               70.0     21.0  ...     21.0   
                Condo/Co-op                   11.0      4.0  ...      5.0   
                Multi-Family (2-4 Unit)        5.0      0.0  ...      0.0   
                Single Family Residential     49.0     12.0  ...     15.0   
                Townhouse                      5.0      5.0  ...      1.0   

                                           2015-08  2022-02  2020-11  2013-10  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential               57.0     44.0     89.0     38.0   
                Condo/Co-op                   57.0     44.0     89.0     38.0   
                Single Family Residential      0.0      0.0      0.0      0.0   
                Townhouse                      0.0      0.0      0.0      0.0   
Zip Code: 98102 All Residential               35.0     29.0    100.0     57.0   
...                                            ...      ...      ...      ...   
Zip Code: 98199 All Residential               37.0     13.0     61.0     76.0   
                Condo/Co-op                    4.0      4.0     13.0     13.0   
                Multi-Family (2-4 Unit)        1.0      0.0      4.0      2.0   
                Single Family Residential 

In [222]:
# Convert the index level 'zipcode' to a regular column
inventorydf.reset_index(level='zipcode', inplace=True)



In [223]:
# Extract zip codes from the existing index and overwrite the index with the modified values
inventorydf.index = inventorydf.index.str.extract(r'(\d{5})', expand=False)
# inventorydf['zipcode'] = inventorydf['zipcode'].str.extract(r'(\d{5})')

# Convert column names to datetime

In [224]:
inventorydf


,zipcode,2018-05,2014-07,2014-11,2014-01,2018-11,2022-03,2020-09,2012-09,2014-06,...,2018-01,2015-08,2022-02,2020-11,2013-10,2015-06,2019-07,2016-08,2022-09,2022-12
property_type,,,,,,,,,,,,,,,,,,,,,
NaN,Zip Code: 98101,61.0,39.0,19.0,19.0,50.0,49.0,130.0,60.0,31.0,...,18.0,57.0,44.0,89.0,38.0,52.0,56.0,34.0,62.0,44.0
NaN,Zip Code: 98101,61.0,39.0,19.0,19.0,50.0,49.0,130.0,60.0,31.0,...,18.0,57.0,44.0,89.0,38.0,52.0,56.0,34.0,62.0,44.0
NaN,Zip Code: 98101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN,Zip Code: 98101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN,Zip Code: 98102,36.0,50.0,33.0,37.0,58.0,31.0,120.0,61.0,51.0,...,20.0,35.0,29.0,100.0,57.0,36.0,97.0,40.0,54.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,Zip Code: 98199,40.0,84.0,39.0,44.0,55.0,18.0,65.0,87.0,70.0,...,21.0,37.0,13.0,61.0,76.0,36.0,73.0,45.0,70.0,33.0
NaN,Zip Code: 98199,7.0,12.0,3.0,1.0,13.0,3.0,11.0,15.0,11.0,...,5.0,4.0,4.0,13.0,13.0,6.0,13.0,7.0,11.0,6.0
NaN,Zip Code: 98199,2.0,5.0,0.0,0.0,1.0,0.0,3.0,2.0,5.0,...,0.0,1.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,1.0


In [217]:
inventorydf.columns = pd.to_datetime(inventorydf.columns, format='%Y-%m')

ValueError: time data "zipcode" doesn't match format "%Y-%m", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [208]:
inventorydf = inventorydf.sort_index(axis=1)

# inventorydf.columns = inventorydf.columns.to_timestamp()
# inventorydf.columns = pd.to_datetime(inventorydf.columns)
# df_sorted = df.sort_index(level='zipcode', axis=0)

TypeError: '<' not supported between instances of 'Period' and 'str'

In [197]:
inventorydf

2012-03-01  2012-04-01  2012-05-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential                  54.0        60.0        53.0   
                Condo/Co-op                      53.0        59.0        52.0   
                Single Family Residential         0.0         0.0         0.0   
                Townhouse                         0.0         0.0         0.0   
Zip Code: 98102 All Residential                  59.0        69.0        62.0   
...                                               ...         ...         ...   
Zip Code: 98199 All Residential                  85.0        83.0        80.0   
                Condo/Co-op                      21.0        21.0        17.0   
                Multi-Family (2-4 Unit)           2.0         0.0         2.0   
                Single Family Residential        58.0        52.0        56.0   
                Townhouse                         4.0        10.0         5.0   

                                           2012-06-01  2012-07-01  2012-08-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential                  60.0        58.0        53.0   
                Condo/Co-op                      59.0        58.0        53.0   
                Single Family Residential         0.0         0.0         0.0   
                Townhouse                         0.0         0.0         0.0   
Zip Code: 98102 All Residential                  77.0        66.0        59.0   
...                                               ...         ...         ...   
Zip Code: 98199 All Residential                  82.0        92.0        82.0   
                Condo/Co-op                       9.0        13.0        11.0   
                Multi-Family (2-4 Unit)           2.0         2.0         2.0   
                Single Family Residential        67.0        69.0        63.0   
                Townhouse                         4.0         8.0         6.0   

                                           2012-09-01  2012-10-01  2012-11-01  \
zipcode         property_type                                                   
Zip Code: 98101 All Residential                  60.0        48.0        43.0   
                Condo/Co-op                      60.0        48.0        43.0   
                Single Family Residential         0.0         0.0         0.0   
                Townhouse                         0.0         0.0         0.0   
Zip Code: 98102 All Residential                  61.0        47.0        51.0   
...                                               ...         ...         ...   
Zip Code: 98199 All Residential                  87.0        73.0        67.0   
                Condo/Co-op                      15.0        16.0        13.0   
                Multi-Family (2-4 Unit)           2.0         2.0         0.0   
                Single Family Residential        59.0        49.0        48.0   
                Townhouse                        11.0         6.0         4.0   

                                           2012-12-01  ...  2023-05-01  \
zipcode         property_type                          ...               
Zip Code: 98101 All Residential                  45.0  ...        59.0   
                Condo/Co-op                      45.0  ...        59.0   
                Single Family Residential         0.0  ...         0.0   
                Townhouse                         0.0  ...         0.0   
Zip Code: 98102 All Residential                  44.0  ...        54.0   
...                                               ...  ...         ...   
Zip Code: 98199 All Residential                  49.0  ...        40.0   
                Condo/Co-op                      10.0  ...         7.0   
                Multi-Family (2-4 Unit)           0.0  ...         0.0   
                Single Family Residential        35.0  ...        2

In [180]:
list(inventorydf.columns)

[Timestamp('2012-03-01 00:00:00'),
 Timestamp('2012-04-01 00:00:00'),
 Timestamp('2012-05-01 00:00:00'),
 Timestamp('2012-06-01 00:00:00'),
 Timestamp('2012-07-01 00:00:00'),
 Timestamp('2012-08-01 00:00:00'),
 Timestamp('2012-09-01 00:00:00'),
 Timestamp('2012-10-01 00:00:00'),
 Timestamp('2012-11-01 00:00:00'),
 Timestamp('2012-12-01 00:00:00'),
 Timestamp('2013-01-01 00:00:00'),
 Timestamp('2013-02-01 00:00:00'),
 Timestamp('2013-03-01 00:00:00'),
 Timestamp('2013-04-01 00:00:00'),
 Timestamp('2013-05-01 00:00:00'),
 Timestamp('2013-06-01 00:00:00'),
 Timestamp('2013-07-01 00:00:00'),
 Timestamp('2013-08-01 00:00:00'),
 Timestamp('2013-09-01 00:00:00'),
 Timestamp('2013-10-01 00:00:00'),
 Timestamp('2013-11-01 00:00:00'),
 Timestamp('2013-12-01 00:00:00'),
 Timestamp('2014-01-01 00:00:00'),
 Timestamp('2014-02-01 00:00:00'),
 Timestamp('2014-03-01 00:00:00'),
 Timestamp('2014-04-01 00:00:00'),
 Timestamp('2014-05-01 00:00:00'),
 Timestamp('2014-06-01 00:00:00'),
 Timestamp('2014-07-

In [198]:
# inventorydf.columns = pd.to_datetime(inventorydf.columns)
inventorydf.columns = inventorydf.columns.to_period('Q')


In [191]:
inventorydf.head()


2012Q1  2012Q2  2012Q2  2012Q2  \
zipcode         property_type                                               
Zip Code: 98101 All Residential              54.0    60.0    53.0    60.0   
                Condo/Co-op                  53.0    59.0    52.0    59.0   
                Single Family Residential     0.0     0.0     0.0     0.0   
                Townhouse                     0.0     0.0     0.0     0.0   
Zip Code: 98102 All Residential              59.0    69.0    62.0    77.0   

                                           2012Q3  2012Q3  2012Q3  2012Q4  \
zipcode         property_type                                               
Zip Code: 98101 All Residential              58.0    53.0    60.0    48.0   
                Condo/Co-op                  58.0    53.0    60.0    48.0   
                Single Family Residential     0.0     0.0     0.0     0.0   
                Townhouse                     0.0     0.0     0.0     0.0   
Zip Code: 98102 All Residential              66.0    59.0    61.0    47.0   

                                           2012Q4  2012Q4  ...  2023Q2  \
zipcode         property_type                              ...           
Zip Code: 98101 All Residential              43.0    45.0  ...    59.0   
                Condo/Co-op                  43.0    45.0  ...    59.0   
                Single Family Residential     0.0     0.0  ...     0.0   
                Townhouse                     0.0     0.0  ...     0.0   
Zip Code: 98102 All Residential              51.0    44.0  ...    54.0   

                                           2023Q2  2023Q3  2023Q3  2023Q3  \
zipcode         property_type                                               
Zip Code: 98101 All Residential              70.0    63.0    67.0    75.0   
                Condo/Co-op                  70.0    63.0    67.0    75.0   
                Single Family Residential     0.0     0.0     0.0     0.0   
                Townhouse                     0.0     0.0     0.0     0.0   
Zip Code: 98102 All Residential              57.0    54.0    66.0    74.0   

                                           2023Q4  2023Q4  2023Q4  2024Q1  \
zipcode         property_type                                               
Zip Code: 98101 All Residential              71.0    59.0    55.0    53.0   
                Condo/Co-op                  71.0    59.0    55.0    53.0   
                Single Family Residential     0.0     0.0     0.0     0.0   
                Townhouse                     0.0     0.0     0.0     0.0   
Zip Code: 98102 All Residential              62.0    47.0    32.0    38.0   

                                           2024Q1  
zipcode         property_type                      
Zip Code: 98101 All Residential              59.0  
                Condo/Co-op                  59.0  
                Single Family Residential     0.0  
                Townhouse                     0.0  
Zip Code: 98102 All Residential              46.0  

[5 rows x 144 columns]

In [199]:
inventorydf = inventorydf.resample('Q', axis=1).mean()

/var/folders/58/j89y2_691mz0dzk3rc7p_08h0000gp/T/ipykernel_42213/3287294964.py:1: FutureWarning: DataFrame.resample with axis=1 is deprecated. Do `frame.T.resample(...)` without axis instead.
  inventorydf = inventorydf.resample('Q', axis=1).mean()


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [173]:
list(inventorydf.columns)

[Period('2012-03', 'M'),
 Period('2012-04', 'M'),
 Period('2012-05', 'M'),
 Period('2012-06', 'M'),
 Period('2012-07', 'M'),
 Period('2012-08', 'M'),
 Period('2012-09', 'M'),
 Period('2012-10', 'M'),
 Period('2012-11', 'M'),
 Period('2012-12', 'M'),
 Period('2013-01', 'M'),
 Period('2013-02', 'M'),
 Period('2013-03', 'M'),
 Period('2013-04', 'M'),
 Period('2013-05', 'M'),
 Period('2013-06', 'M'),
 Period('2013-07', 'M'),
 Period('2013-08', 'M'),
 Period('2013-09', 'M'),
 Period('2013-10', 'M'),
 Period('2013-11', 'M'),
 Period('2013-12', 'M'),
 Period('2014-01', 'M'),
 Period('2014-02', 'M'),
 Period('2014-03', 'M'),
 Period('2014-04', 'M'),
 Period('2014-05', 'M'),
 Period('2014-06', 'M'),
 Period('2014-07', 'M'),
 Period('2014-08', 'M'),
 Period('2014-09', 'M'),
 Period('2014-10', 'M'),
 Period('2014-11', 'M'),
 Period('2014-12', 'M'),
 Period('2015-01', 'M'),
 Period('2015-02', 'M'),
 Period('2015-03', 'M'),
 Period('2015-04', 'M'),
 Period('2015-05', 'M'),
 Period('2015-06', 'M'),


In [175]:
inventorydf.columns = inventorydf[
'2018-01', 'M'),
 '2018-02', 'M'),
 '2018-03', 'M'),
'2018-04', 'M'),
 Period('2018-05', 'M'),
 Period('2018-06', 'M'),
 Period('2018-07', 'M'),
 Period('2018-08', 'M'),
 Period('2018-09', 'M'),
 Period('2018-10', 'M'),
 Period('2018-11', 'M'),
 Period('2018-12', 'M'),
 Period('2019-01', 'M'),
 Period('2019-02', 'M'),
 Period('2019-03', 'M'),
 Period('2019-04', 'M'),
 Period('2019-05', 'M'),
 Period('2019-06', 'M'),
 Period('2019-07', 'M'),
 Period('2019-08', 'M'),
 Period('2019-09', 'M'),
 Period('2019-10', 'M'),
 Period('2019-11', 'M'),
 Period('2019-12', 'M'),
 Period('2020-01', 'M'),
 Period('2020-02', 'M'),
 Period('2020-03', 'M'),
 Period('2020-04', 'M'),
 Period('2020-05', 'M'),
 Period('2020-06', 'M'),
 Period('2020-07', 'M'),
 Period('2020-08', 'M'),
 Period('2020-09', 'M'),
 Period('2020-10', 'M'),
 Period('2020-11', 'M'),
 Period('2020-12', 'M'),
 Period('2021-01', 'M'),
 Period('2021-02', 'M'),
 Period('2021-03', 'M'),
 Period('2021-04', 'M'),
 Period('2021-05', 'M'),
 Period('2021-06', 'M'),
 Period('2021-07', 'M'),
 Period('2021-08', 'M'),
 Period('2021-09', 'M'),
 Period('2021-10', 'M'),
 Period('2021-11', 'M'),
 Period('2021-12', 'M'),
 Period('2022-01', 'M'),
 Period('2022-02', 'M'),
 Period('2022-03', 'M'),
 Period('2022-04', 'M'),
 Period('2022-05', 'M'),
 Period('2022-06', 'M'),
 Period('2022-07', 'M'),
 Period('2022-08', 'M'),
 Period('2022-09', 'M'),
 Period('2022-10', 'M'),
 Period('2022-11', 'M'),
 Period('2022-12', 'M')]

NameError: name 'Period' is not defined

# Section Rental Filtering

In [98]:
filepath_metrodata2 = "../data_sources/king_county_rental.csv.zip"
rental_csv = pd.read_csv(filepath_metrodata2)
rental_csv.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
0,99534,145,98052,zip,WA,WA,Redmond,"Seattle-Tacoma-Bellevue, WA",King County,NaN,...,2389.406076,2407.413148,2417.732657,2407.582999,2386.135635,2362.543944,2342.793871,2331.308076,2354.321273,2384.353490
1,99565,815,98105,zip,WA,WA,Seattle,"Seattle-Tacoma-Bellevue, WA",King County,NaN,...,2286.436079,2315.705835,2340.219275,2356.803648,2350.684313,2328.200090,2298.415649,2304.796452,2319.678014,2328.472768
2,99559,829,98092,zip,WA,WA,NaN,"Seattle-Tacoma-Bellevue, WA",King County,NaN,...,1989.343131,2022.142233,2028.593958,2030.784220,2029.335833,2011.968245,1998.949459,1974.576889,1987.518708,2003.019048
3,99575,857,98115,zip,WA,WA,Seattle,"Seattle-Tacoma-Bellevue, WA",King County,NaN,...,2181.871488,2199.526987,2218.435755,2224.014098,2233.063028,2216.982879,2211.136956,2190.867984,2193.711036,2204.760694
4,99508,862,98023,zip,WA,WA,Federal Way,"Seattle-Tacoma-Bellevue, WA",King County,NaN,...,1910.012848,1935.575801,1964.565702,1962.232322,1943.192864,1936.072215,1927.145717,1938.080172,1903.159434,1912.334649


In [99]:
rental_csv.info


<bound method DataFrame.info of     RegionID  SizeRank  RegionName RegionType StateName State           City  \
0      99534       145       98052        zip        WA    WA        Redmond   
1      99565       815       98105        zip        WA    WA        Seattle   
2      99559       829       98092        zip        WA    WA            NaN   
3      99575       857       98115        zip        WA    WA        Seattle   
4      99508       862       98023        zip        WA    WA    Federal Way   
..       ...       ...         ...        ...       ...   ...            ...   
61     99564      8196       98104        zip        WA    WA        Seattle   
62     99539      8461       98057        zip        WA    WA         Renton   
63     99546      8621       98065        zip        WA    WA     Snoqualmie   
64     99498     11601       98010        zip        WA    WA  Black Diamond   
65     99531     12968       98047        zip        WA    WA        Pacific   

       

In [100]:
#relevatn zips taken from readme fo zipcodes in seattle

relevantzips = [
    98101,
98102,
98103,
98104,
98105,
98106,
98107,
98108,
98109,
98110,
98111,
98112,
98113,
98114,
98115,
98116,
98117,
98118,
98119,
98121,
98122,
98124,
98125,
98126,
98127,
98129,
98131,
98133,
98134,
98136,
98138,
98139,
98141,
98144,
98145,
98146,
98148,
98154,
98155,
98158,
98160,
98161,
98164,
98165,
98166,
98168,
98170,
98174,
98175,
98177,
98178,
98181,
98185,
98188,
98190,
98191,
98194,
98195,
98198,
98199]
relevantzips

[98101,
 98102,
 98103,
 98104,
 98105,
 98106,
 98107,
 98108,
 98109,
 98110,
 98111,
 98112,
 98113,
 98114,
 98115,
 98116,
 98117,
 98118,
 98119,
 98121,
 98122,
 98124,
 98125,
 98126,
 98127,
 98129,
 98131,
 98133,
 98134,
 98136,
 98138,
 98139,
 98141,
 98144,
 98145,
 98146,
 98148,
 98154,
 98155,
 98158,
 98160,
 98161,
 98164,
 98165,
 98166,
 98168,
 98170,
 98174,
 98175,
 98177,
 98178,
 98181,
 98185,
 98188,
 98190,
 98191,
 98194,
 98195,
 98198,
 98199]

In [101]:
len(relevantzips)

60

In [102]:
seattle_zips = rental_csv[rental_csv['RegionName'].isin(relevantzips)]

#seattle_zips contains data for 25 zipcodes in seattle. the relevant_zips contains 41 zipcodes
#From now on, we will use the data for these 25 zipcodes in rental_zipcode_list
seattle_zips['RegionName'].unique()
rental_zipcode_list = [98105, 98115, 98103, 98133, 98118, 98125, 98122, 98117, 98155,
       98198, 98168, 98109, 98146, 98144, 98188, 98107, 98106, 98178,
       98116, 98108, 98112, 98119, 98126, 98199, 98102, 98166, 98121,
       98136, 98101, 98104]



In [103]:
col_list= list(seattle_zips.columns)
(col_list)

['RegionID',
 'SizeRank',
 'RegionName',
 'RegionType',
 'StateName',
 'State',
 'City',
 'Metro',
 'CountyName',
 '2015-01-31',
 '2015-02-28',
 '2015-03-31',
 '2015-04-30',
 '2015-05-31',
 '2015-06-30',
 '2015-07-31',
 '2015-08-31',
 '2015-09-30',
 '2015-10-31',
 '2015-11-30',
 '2015-12-31',
 '2016-01-31',
 '2016-02-29',
 '2016-03-31',
 '2016-04-30',
 '2016-05-31',
 '2016-06-30',
 '2016-07-31',
 '2016-08-31',
 '2016-09-30',
 '2016-10-31',
 '2016-11-30',
 '2016-12-31',
 '2017-01-31',
 '2017-02-28',
 '2017-03-31',
 '2017-04-30',
 '2017-05-31',
 '2017-06-30',
 '2017-07-31',
 '2017-08-31',
 '2017-09-30',
 '2017-10-31',
 '2017-11-30',
 '2017-12-31',
 '2018-01-31',
 '2018-02-28',
 '2018-03-31',
 '2018-04-30',
 '2018-05-31',
 '2018-06-30',
 '2018-07-31',
 '2018-08-31',
 '2018-09-30',
 '2018-10-31',
 '2018-11-30',
 '2018-12-31',
 '2019-01-31',
 '2019-02-28',
 '2019-03-31',
 '2019-04-30',
 '2019-05-31',
 '2019-06-30',
 '2019-07-31',
 '2019-08-31',
 '2019-09-30',
 '2019-10-31',
 '2019-11-30',
 

In [104]:
# clean data for zipcode and city if we need it, then mean rental price per zipcode for every month
#Here is where we can filter for certain months or transfer this data into quarters 
#Much of the data for 2015 is incomplete
seattle_zips = seattle_zips[[
 'RegionName',
 'City',
 '2015-01-31',
 '2015-02-28',
 '2015-03-31',
 '2015-04-30',
 '2015-05-31',
 '2015-06-30',
 '2015-07-31',
 '2015-08-31',
 '2015-09-30',
 '2015-10-31',
 '2015-11-30',
 '2015-12-31',
 '2016-01-31',
 '2016-02-29',
 '2016-03-31',
 '2016-04-30',
 '2016-05-31',
 '2016-06-30',
 '2016-07-31',
 '2016-08-31',
 '2016-09-30',
 '2016-10-31',
 '2016-11-30',
 '2016-12-31',
 '2017-01-31',
 '2017-02-28',
 '2017-03-31',
 '2017-04-30',
 '2017-05-31',
 '2017-06-30',
 '2017-07-31',
 '2017-08-31',
 '2017-09-30',
 '2017-10-31',
 '2017-11-30',
 '2017-12-31',
 '2018-01-31',
 '2018-02-28',
 '2018-03-31',
 '2018-04-30',
 '2018-05-31',
 '2018-06-30',
 '2018-07-31',
 '2018-08-31',
 '2018-09-30',
 '2018-10-31',
 '2018-11-30',
 '2018-12-31',
 '2019-01-31',
 '2019-02-28',
 '2019-03-31',
 '2019-04-30',
 '2019-05-31',
 '2019-06-30',
 '2019-07-31',
 '2019-08-31',
 '2019-09-30',
 '2019-10-31',
 '2019-11-30',
 '2019-12-31',
 '2020-01-31',
 '2020-02-29',
 '2020-03-31',
 '2020-04-30',
 '2020-05-31',
 '2020-06-30',
 '2020-07-31',
 '2020-08-31',
 '2020-09-30',
 '2020-10-31',
 '2020-11-30',
 '2020-12-31',
 '2021-01-31',
 '2021-02-28',
 '2021-03-31',
 '2021-04-30',
 '2021-05-31',
 '2021-06-30',
 '2021-07-31',
 '2021-08-31',
 '2021-09-30',
 '2021-10-31',
 '2021-11-30',
 '2021-12-31',
 '2022-01-31',
 '2022-02-28',
 '2022-03-31',
 '2022-04-30',
 '2022-05-31',
 '2022-06-30',
 '2022-07-31',
 '2022-08-31',
 '2022-09-30',
 '2022-10-31',
 '2022-11-30',
 '2022-12-31',
 '2023-01-31',
 '2023-02-28',
 '2023-03-31',
 '2023-04-30',
 '2023-05-31',
 '2023-06-30',
 '2023-07-31',
 '2023-08-31',
 '2023-09-30',
 '2023-10-31',
 '2023-11-30',
 '2023-12-31',
 '2024-01-31',
 '2024-02-29']]

In [105]:
# seattle_zips = pd.to_datetime(seattle_zips.columns[2:])
seattle_zips

,RegionName,City,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,...,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
1,98105,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2286.436079,2315.705835,2340.219275,2356.803648,2350.684313,2328.200090,2298.415649,2304.796452,2319.678014,2328.472768
3,98115,Seattle,NaN,1510.025231,1525.516126,1528.994058,1553.590060,1567.551558,1574.868723,1569.762779,...,2181.871488,2199.526987,2218.435755,2224.014098,2233.063028,2216.982879,2211.136956,2190.867984,2193.711036,2204.760694
5,98103,Seattle,1458.092888,1462.702361,1475.065810,1487.273520,1500.935227,1530.842226,1556.451838,1582.614332,...,2124.198302,2139.808211,2163.042594,2163.185572,2159.909977,2133.551853,2116.569993,2109.871054,2109.282055,2125.706081
9,98133,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1800.355117,1812.633435,1825.123148,1846.613225,1837.038918,1818.714400,1817.030356,1807.532613,1821.429759,1843.687629
10,98118,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1738.934560,1745.086948,1748.460216,1746.712016,1750.472336,1746.940618,1737.354735,1760.572859,1769.899750,1783.338109
11,98125,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1937.087540,1942.022441,1957.586585,1954.097476,1987.494911,1991.135278,1963.390333,1938.268652,1933.499766,1958.713507
14,98122,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2028.082751,2041.982668,2054.447483,2063.676180,2067.088061,2054.904296,2039.966725,2014.932682,2020.375608,2028.117279
21,98117,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2111.060292,2124.573363,2133.156681,2134.609451,2160.443746,2139.113660,2164.603611,2179.853561,2182.908575,2152.239683
24,98155,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2120.921028,2150.699564,2179.310162,2180.367673,2174.761381,2140.492173,2124.185061,2113.704995,2143.416903,2164.419192
25,98198,Des Moines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1977.568631,1951.595814,1953.587358,1933.851810,1969.590121,1971.858229,1977.235448,1985.021246,1993.554096,2004.558333


In [112]:
seattle_zips.index.dtype
seattle_rentals_by_zipcodes = seattle_zips.copy()

In [113]:
seattle_rentals_by_zipcodes


,RegionName,City,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,...,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
1,98105,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2286.436079,2315.705835,2340.219275,2356.803648,2350.684313,2328.200090,2298.415649,2304.796452,2319.678014,2328.472768
3,98115,Seattle,NaN,1510.025231,1525.516126,1528.994058,1553.590060,1567.551558,1574.868723,1569.762779,...,2181.871488,2199.526987,2218.435755,2224.014098,2233.063028,2216.982879,2211.136956,2190.867984,2193.711036,2204.760694
5,98103,Seattle,1458.092888,1462.702361,1475.065810,1487.273520,1500.935227,1530.842226,1556.451838,1582.614332,...,2124.198302,2139.808211,2163.042594,2163.185572,2159.909977,2133.551853,2116.569993,2109.871054,2109.282055,2125.706081
9,98133,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1800.355117,1812.633435,1825.123148,1846.613225,1837.038918,1818.714400,1817.030356,1807.532613,1821.429759,1843.687629
10,98118,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1738.934560,1745.086948,1748.460216,1746.712016,1750.472336,1746.940618,1737.354735,1760.572859,1769.899750,1783.338109
11,98125,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1937.087540,1942.022441,1957.586585,1954.097476,1987.494911,1991.135278,1963.390333,1938.268652,1933.499766,1958.713507
14,98122,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2028.082751,2041.982668,2054.447483,2063.676180,2067.088061,2054.904296,2039.966725,2014.932682,2020.375608,2028.117279
21,98117,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2111.060292,2124.573363,2133.156681,2134.609451,2160.443746,2139.113660,2164.603611,2179.853561,2182.908575,2152.239683
24,98155,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2120.921028,2150.699564,2179.310162,2180.367673,2174.761381,2140.492173,2124.185061,2113.704995,2143.416903,2164.419192
25,98198,Des Moines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1977.568631,1951.595814,1953.587358,1933.851810,1969.590121,1971.858229,1977.235448,1985.021246,1993.554096,2004.558333


In [114]:
seattle_rentals_by_zipcodes.reset_index(inplace=True)
seattle_rentals_by_zipcodes = seattle_rentals_by_zipcodes.drop(columns='index')


In [112]:
# Save the data for the rental_price_by_zipcode_property_type of homes in seattle zipcodes by month
seattle_rentals_by_zipcodes.to_csv("../data_frames/seattle_rentals_by_zipcodes.csv", index=True)

In [115]:
rentals_2018_and_after = seattle_rentals_by_zipcodes.set_index('RegionName', inplace=False)


In [116]:
rentals_2018_and_after 

,City,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,...,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
RegionName,,,,,,,,,,,,,,,,,,,,,
98105,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2286.436079,2315.705835,2340.219275,2356.803648,2350.684313,2328.200090,2298.415649,2304.796452,2319.678014,2328.472768
98115,Seattle,NaN,1510.025231,1525.516126,1528.994058,1553.590060,1567.551558,1574.868723,1569.762779,1592.647437,...,2181.871488,2199.526987,2218.435755,2224.014098,2233.063028,2216.982879,2211.136956,2190.867984,2193.711036,2204.760694
98103,Seattle,1458.092888,1462.702361,1475.065810,1487.273520,1500.935227,1530.842226,1556.451838,1582.614332,1584.987217,...,2124.198302,2139.808211,2163.042594,2163.185572,2159.909977,2133.551853,2116.569993,2109.871054,2109.282055,2125.706081
98133,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1800.355117,1812.633435,1825.123148,1846.613225,1837.038918,1818.714400,1817.030356,1807.532613,1821.429759,1843.687629
98118,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1738.934560,1745.086948,1748.460216,1746.712016,1750.472336,1746.940618,1737.354735,1760.572859,1769.899750,1783.338109
98125,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1937.087540,1942.022441,1957.586585,1954.097476,1987.494911,1991.135278,1963.390333,1938.268652,1933.499766,1958.713507
98122,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2028.082751,2041.982668,2054.447483,2063.676180,2067.088061,2054.904296,2039.966725,2014.932682,2020.375608,2028.117279
98117,Seattle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2111.060292,2124.573363,2133.156681,2134.609451,2160.443746,2139.113660,2164.603611,2179.853561,2182.908575,2152.239683
98155,Shoreline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2120.921028,2150.699564,2179.310162,2180.367673,2174.761381,2140.492173,2124.185061,2113.704995,2143.416903,2164.419192


In [117]:
# seattle_rentals_by_zipcodes for 2018-present
rentals_2018_and_after = rentals_2018_and_after.loc[:, '2018-01-31':]


rentals_2018_and_after

,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31,2024-02-29
RegionName,,,,,,,,,,,,,,,,,,,,,
98105,1852.385713,1848.739328,1865.245256,1908.863772,1927.991463,1956.577881,1956.049413,1965.174201,1964.853072,1935.224261,...,2286.436079,2315.705835,2340.219275,2356.803648,2350.684313,2328.200090,2298.415649,2304.796452,2319.678014,2328.472768
98115,1757.969216,1767.311466,1779.947620,1788.331376,1817.381904,1824.310880,1830.082528,1829.771119,1832.029909,1817.682651,...,2181.871488,2199.526987,2218.435755,2224.014098,2233.063028,2216.982879,2211.136956,2190.867984,2193.711036,2204.760694
98103,1756.543138,1764.710110,1789.244197,1786.118929,1791.794669,1799.133719,1816.494305,1821.311801,1824.396477,1804.434316,...,2124.198302,2139.808211,2163.042594,2163.185572,2159.909977,2133.551853,2116.569993,2109.871054,2109.282055,2125.706081
98133,1427.322972,1434.378203,1427.118038,1435.253116,1441.060402,1455.775157,1473.216841,1476.360181,1479.436761,1467.551851,...,1800.355117,1812.633435,1825.123148,1846.613225,1837.038918,1818.714400,1817.030356,1807.532613,1821.429759,1843.687629
98118,NaN,NaN,1398.541862,1427.220685,1426.746158,1411.180963,1417.703419,1432.305903,1447.997850,1449.824882,...,1738.934560,1745.086948,1748.460216,1746.712016,1750.472336,1746.940618,1737.354735,1760.572859,1769.899750,1783.338109
98125,1530.351429,1533.247650,1542.801778,1539.214532,1568.509796,1573.332985,1584.445072,1592.356557,1589.955867,1564.654107,...,1937.087540,1942.022441,1957.586585,1954.097476,1987.494911,1991.135278,1963.390333,1938.268652,1933.499766,1958.713507
98122,1773.813812,1785.300934,1801.406426,1816.991574,1825.824457,1852.095022,1856.482541,1861.624426,1844.434079,1819.831390,...,2028.082751,2041.982668,2054.447483,2063.676180,2067.088061,2054.904296,2039.966725,2014.932682,2020.375608,2028.117279
98117,1683.620738,1684.873889,1691.777125,1717.527534,1752.070327,1737.107794,1728.024880,1741.891468,1759.761018,1747.546549,...,2111.060292,2124.573363,2133.156681,2134.609451,2160.443746,2139.113660,2164.603611,2179.853561,2182.908575,2152.239683
98155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2120.921028,2150.699564,2179.310162,2180.367673,2174.761381,2140.492173,2124.185061,2113.704995,2143.416903,2164.419192


In [122]:
rentals_2018_and_after.dtypes

2018-01-31    float64
2018-02-28    float64
2018-03-31    float64
2018-04-30    float64
2018-05-31    float64
               ...   
2023-10-31    float64
2023-11-30    float64
2023-12-31    float64
2024-01-31    float64
2024-02-29    float64
Length: 74, dtype: object

In [121]:
rentals_2018_and_after.columns = pd.to_datetime(rentals_2018_and_after.columns)

In [123]:

rentals_by_quarters = rentals_2018_and_after.resample('Q', axis=1).mean()

/var/folders/58/j89y2_691mz0dzk3rc7p_08h0000gp/T/ipykernel_42213/1977689564.py:1: FutureWarning: DataFrame.resample with axis=1 is deprecated. Do `frame.T.resample(...)` without axis instead.
  rentals_by_quarters = rentals_2018_and_after.resample('Q', axis=1).mean()


In [124]:
rentals_by_quarters

,2018-03-31,2018-06-30,2018-09-30,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,2020-03-31,2020-06-30,...,2021-12-31,2022-03-31,2022-06-30,2022-09-30,2022-12-31,2023-03-31,2023-06-30,2023-09-30,2023-12-31,2024-03-31
RegionName,,,,,,,,,,,,,,,,,,,,,
98105,1855.456766,1931.144372,1962.025562,1909.126985,1923.065864,1982.204503,2007.859793,1969.179699,1988.221904,2010.817096,...,2125.978628,2161.859984,2246.265731,2293.322852,2276.521256,2241.202231,2287.727062,2349.235745,2310.470730,2324.075391
98115,1768.409434,1810.008053,1830.627852,1807.416822,1802.423457,1854.224565,1881.106350,1875.036652,1898.948429,1904.345494,...,2046.299753,2057.797862,2135.322509,2192.493070,2180.195272,2156.179955,2181.173666,2225.170960,2206.329273,2199.235865
98103,1770.165815,1792.349106,1820.734195,1789.206306,1793.185150,1825.717956,1864.735357,1857.160138,1887.419868,1888.426540,...,1953.612783,1990.817499,2065.320524,2121.080646,2073.446859,2061.920272,2118.133761,2162.046048,2119.997633,2117.494068
98133,1429.606404,1444.029558,1476.337927,1457.967321,1457.826854,1498.782832,1509.931683,1524.821405,1537.460382,1557.503190,...,1649.693326,1686.310343,1748.956445,1824.585411,1796.472873,1799.184064,1813.386750,1836.258430,1814.425790,1832.558694
98118,1398.541862,1421.715935,1432.669058,1433.458839,1428.795034,1488.327257,1508.140684,1504.201098,1506.445181,1538.633553,...,1633.498159,1638.501074,1701.164662,1726.100438,1742.698200,1758.789252,1740.845686,1748.548189,1748.289404,1776.618930
98125,1535.466952,1560.352438,1588.919165,1558.985749,1570.263987,1580.204676,1619.149553,1605.317286,1617.899681,1652.799382,...,1767.695362,1797.215062,1907.705408,1927.827563,1897.663381,1871.313880,1928.079361,1966.392990,1964.264754,1946.106636
98122,1786.840390,1831.637018,1854.180349,1810.498746,1840.079384,1859.305049,1909.594382,1898.938525,1916.011883,1919.330131,...,1958.901355,1965.208220,2028.024089,2090.526076,2019.541023,2000.433469,2030.008298,2061.737241,2036.601234,2024.246444
98117,1686.757251,1735.568552,1743.225789,1724.823392,1711.148542,1769.306009,1793.860773,1779.729888,1816.595445,1808.747832,...,1954.217882,2014.398975,2076.303212,2116.137213,2071.517101,2048.039332,2109.798017,2142.736626,2161.190278,2167.574129
98155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1769.244996,1758.318764,...,1928.415517,1991.703371,2058.669709,2104.471184,2074.759832,2058.051248,2110.080284,2178.146406,2126.127410,2153.918048


In [125]:
rentals_by_quarters.columns = rentals_by_quarters.columns.to_period('Q').strftime('%YQ%q')


In [127]:
list(rentals_by_quarters.columns)

['2018Q1',
 '2018Q2',
 '2018Q3',
 '2018Q4',
 '2019Q1',
 '2019Q2',
 '2019Q3',
 '2019Q4',
 '2020Q1',
 '2020Q2',
 '2020Q3',
 '2020Q4',
 '2021Q1',
 '2021Q2',
 '2021Q3',
 '2021Q4',
 '2022Q1',
 '2022Q2',
 '2022Q3',
 '2022Q4',
 '2023Q1',
 '2023Q2',
 '2023Q3',
 '2023Q4',
 '2024Q1']

In [128]:
rentals_by_quarters = rentals_by_quarters[[
'2018Q1',
 '2018Q2',
 '2018Q3',
 '2018Q4',
 '2019Q1',
 '2019Q2',
 '2019Q3',
 '2019Q4',
 '2020Q1',
 '2020Q2',
 '2020Q3',
 '2020Q4',
 '2021Q1',
 '2021Q2',
 '2021Q3',
 '2021Q4',
 '2022Q1',
 '2022Q2',
 '2022Q3',
 '2022Q4',
]]
rentals_by_quarters

,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
RegionName,,,,,,,,,,,,,,,,,,,,
98105,1855.456766,1931.144372,1962.025562,1909.126985,1923.065864,1982.204503,2007.859793,1969.179699,1988.221904,2010.817096,1987.144514,1876.026827,1851.214034,1950.982390,2093.646120,2125.978628,2161.859984,2246.265731,2293.322852,2276.521256
98115,1768.409434,1810.008053,1830.627852,1807.416822,1802.423457,1854.224565,1881.106350,1875.036652,1898.948429,1904.345494,1909.727892,1828.118360,1792.959255,1861.877012,2007.702396,2046.299753,2057.797862,2135.322509,2192.493070,2180.195272
98103,1770.165815,1792.349106,1820.734195,1789.206306,1793.185150,1825.717956,1864.735357,1857.160138,1887.419868,1888.426540,1885.380880,1806.813093,1762.255015,1851.725959,1944.203126,1953.612783,1990.817499,2065.320524,2121.080646,2073.446859
98133,1429.606404,1444.029558,1476.337927,1457.967321,1457.826854,1498.782832,1509.931683,1524.821405,1537.460382,1557.503190,1557.577956,1541.975615,1520.663868,1527.685692,1637.589978,1649.693326,1686.310343,1748.956445,1824.585411,1796.472873
98118,1398.541862,1421.715935,1432.669058,1433.458839,1428.795034,1488.327257,1508.140684,1504.201098,1506.445181,1538.633553,1541.912816,1435.536701,1420.589363,1489.031878,1609.185333,1633.498159,1638.501074,1701.164662,1726.100438,1742.698200
98125,1535.466952,1560.352438,1588.919165,1558.985749,1570.263987,1580.204676,1619.149553,1605.317286,1617.899681,1652.799382,1655.093674,1609.796829,1569.927009,1607.213137,1734.485010,1767.695362,1797.215062,1907.705408,1927.827563,1897.663381
98122,1786.840390,1831.637018,1854.180349,1810.498746,1840.079384,1859.305049,1909.594382,1898.938525,1916.011883,1919.330131,1870.358964,1690.068264,1628.107029,1750.532750,1941.096321,1958.901355,1965.208220,2028.024089,2090.526076,2019.541023
98117,1686.757251,1735.568552,1743.225789,1724.823392,1711.148542,1769.306009,1793.860773,1779.729888,1816.595445,1808.747832,1840.144983,1785.753120,1774.969235,1821.052481,1917.852293,1954.217882,2014.398975,2076.303212,2116.137213,2071.517101
98155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1769.244996,1758.318764,1806.743929,1784.749747,1780.364981,1835.943147,1879.780289,1928.415517,1991.703371,2058.669709,2104.471184,2074.759832


In [129]:
rentals_by_quarters = rentals_by_quarters.rename_axis('Zipcode')
rentals_by_quarters

,2018Q1,2018Q2,2018Q3,2018Q4,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,2020Q2,2020Q3,2020Q4,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4
Zipcode,,,,,,,,,,,,,,,,,,,,
98105,1855.456766,1931.144372,1962.025562,1909.126985,1923.065864,1982.204503,2007.859793,1969.179699,1988.221904,2010.817096,1987.144514,1876.026827,1851.214034,1950.982390,2093.646120,2125.978628,2161.859984,2246.265731,2293.322852,2276.521256
98115,1768.409434,1810.008053,1830.627852,1807.416822,1802.423457,1854.224565,1881.106350,1875.036652,1898.948429,1904.345494,1909.727892,1828.118360,1792.959255,1861.877012,2007.702396,2046.299753,2057.797862,2135.322509,2192.493070,2180.195272
98103,1770.165815,1792.349106,1820.734195,1789.206306,1793.185150,1825.717956,1864.735357,1857.160138,1887.419868,1888.426540,1885.380880,1806.813093,1762.255015,1851.725959,1944.203126,1953.612783,1990.817499,2065.320524,2121.080646,2073.446859
98133,1429.606404,1444.029558,1476.337927,1457.967321,1457.826854,1498.782832,1509.931683,1524.821405,1537.460382,1557.503190,1557.577956,1541.975615,1520.663868,1527.685692,1637.589978,1649.693326,1686.310343,1748.956445,1824.585411,1796.472873
98118,1398.541862,1421.715935,1432.669058,1433.458839,1428.795034,1488.327257,1508.140684,1504.201098,1506.445181,1538.633553,1541.912816,1435.536701,1420.589363,1489.031878,1609.185333,1633.498159,1638.501074,1701.164662,1726.100438,1742.698200
98125,1535.466952,1560.352438,1588.919165,1558.985749,1570.263987,1580.204676,1619.149553,1605.317286,1617.899681,1652.799382,1655.093674,1609.796829,1569.927009,1607.213137,1734.485010,1767.695362,1797.215062,1907.705408,1927.827563,1897.663381
98122,1786.840390,1831.637018,1854.180349,1810.498746,1840.079384,1859.305049,1909.594382,1898.938525,1916.011883,1919.330131,1870.358964,1690.068264,1628.107029,1750.532750,1941.096321,1958.901355,1965.208220,2028.024089,2090.526076,2019.541023
98117,1686.757251,1735.568552,1743.225789,1724.823392,1711.148542,1769.306009,1793.860773,1779.729888,1816.595445,1808.747832,1840.144983,1785.753120,1774.969235,1821.052481,1917.852293,1954.217882,2014.398975,2076.303212,2116.137213,2071.517101
98155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1769.244996,1758.318764,1806.743929,1784.749747,1780.364981,1835.943147,1879.780289,1928.415517,1991.703371,2058.669709,2104.471184,2074.759832


In [130]:
# Save the data for the rental_price_by_zipcode of homes in seattle zipcodes by quarter
rentals_by_quarters.to_csv("../data_frames/rentals_by_quarters.csv", index=True)

# Section Census Beginning

In [40]:
# filepath_census2022 = "../greg/acsdp1y_5y_2010_2022/ACSDP1Y2010.DP03-Column-Metadata.csv"
census2022 = pd.read_csv("../greg/acsdp1y_5y_2010_2022/ACSDP1Y2010.DP03-Column-Metadata.csv")
census2022



,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
3,DP03_0001M,Estimate Margin of Error!!EMPLOYMENT STATUS!!P...
4,DP03_0001PE,Percent!!EMPLOYMENT STATUS!!Population 16 year...
...,...,...
545,DP03_0136PM,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
546,DP03_0137E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
547,DP03_0137M,Estimate Margin of Error!!PERCENTAGE OF FAMILI...
548,DP03_0137PE,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHO...


In [41]:
columns_in_census = list(census2022['Column Name'])

In [42]:
columns_in_census

['GEO_ID',
 'NAME',
 'DP03_0001E',
 'DP03_0001M',
 'DP03_0001PE',
 'DP03_0001PM',
 'DP03_0002E',
 'DP03_0002M',
 'DP03_0002PE',
 'DP03_0002PM',
 'DP03_0003E',
 'DP03_0003M',
 'DP03_0003PE',
 'DP03_0003PM',
 'DP03_0004E',
 'DP03_0004M',
 'DP03_0004PE',
 'DP03_0004PM',
 'DP03_0005E',
 'DP03_0005M',
 'DP03_0005PE',
 'DP03_0005PM',
 'DP03_0006E',
 'DP03_0006M',
 'DP03_0006PE',
 'DP03_0006PM',
 'DP03_0007E',
 'DP03_0007M',
 'DP03_0007PE',
 'DP03_0007PM',
 'DP03_0008E',
 'DP03_0008M',
 'DP03_0008PE',
 'DP03_0008PM',
 'DP03_0009E',
 'DP03_0009M',
 'DP03_0009PE',
 'DP03_0009PM',
 'DP03_0010E',
 'DP03_0010M',
 'DP03_0010PE',
 'DP03_0010PM',
 'DP03_0011E',
 'DP03_0011M',
 'DP03_0011PE',
 'DP03_0011PM',
 'DP03_0012E',
 'DP03_0012M',
 'DP03_0012PE',
 'DP03_0012PM',
 'DP03_0013E',
 'DP03_0013M',
 'DP03_0013PE',
 'DP03_0013PM',
 'DP03_0014E',
 'DP03_0014M',
 'DP03_0014PE',
 'DP03_0014PM',
 'DP03_0015E',
 'DP03_0015M',
 'DP03_0015PE',
 'DP03_0015PM',
 'DP03_0016E',
 'DP03_0016M',
 'DP03_0016PE',
 'DP

In [43]:
descriptionsdf = census2022.copy()

In [44]:
descriptionsdf

,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
3,DP03_0001M,Estimate Margin of Error!!EMPLOYMENT STATUS!!P...
4,DP03_0001PE,Percent!!EMPLOYMENT STATUS!!Population 16 year...
...,...,...
545,DP03_0136PM,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
546,DP03_0137E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
547,DP03_0137M,Estimate Margin of Error!!PERCENTAGE OF FAMILI...
548,DP03_0137PE,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHO...


In [49]:
descriptionsdf = descriptionsdf.T


In [50]:
descriptionsdf

,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
3,DP03_0001M,Estimate Margin of Error!!EMPLOYMENT STATUS!!P...
4,DP03_0001PE,Percent!!EMPLOYMENT STATUS!!Population 16 year...
...,...,...
545,DP03_0136PM,Percent Margin of Error!!PERCENTAGE OF FAMILIE...
546,DP03_0137E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
547,DP03_0137M,Estimate Margin of Error!!PERCENTAGE OF FAMILI...
548,DP03_0137PE,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHO...


In [48]:
for column in descriptionsdf.columns:
    if "Percent" in column:
        descriptionsdf = descriptionsdf.drop(columns = column)

TypeError: argument of type 'int' is not iterable

In [53]:
descriptionsdf = descriptionsdf[~descriptionsdf['Label'].str.contains('Percent')]



In [57]:
descriptionsdf

,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
3,DP03_0001M,Estimate Margin of Error!!EMPLOYMENT STATUS!!P...
6,DP03_0002E,Estimate!!EMPLOYMENT STATUS!!In labor force
...,...,...
539,DP03_0135M,Estimate Margin of Error!!PERCENTAGE OF FAMILI...
542,DP03_0136E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
543,DP03_0136M,Estimate Margin of Error!!PERCENTAGE OF FAMILI...
546,DP03_0137E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...


In [70]:
descriptionsdf = descriptionsdf[~descriptionsdf['Label'].str.contains('COMMUTING')]

In [68]:
descriptions_in_census = list(descriptionsdf['Label'])

In [71]:
descriptions_in_census 

['Geography',
 'Geographic Area Name',
 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over',
 'Estimate!!EMPLOYMENT STATUS!!In labor force',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Employed',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Unemployed',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Armed Forces',
 'Estimate!!EMPLOYMENT STATUS!!Not in labor force',
 'Estimate!!EMPLOYMENT STATUS!!Civilian labor force',
 'Estimate!!EMPLOYMENT STATUS!!Females 16 years and over',
 'Estimate!!EMPLOYMENT STATUS!!In labor force',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force',
 'Estimate!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Employed',
 'Estimate!!EMPLOYMENT STATUS!!Own children under 6 years',
 'Estimate!!EMPLOYMENT STATUS!!All parents in family in labor force',
 'Estimate!!EMPLOYMENT STATUS!!Own children 6 to 17 years',
 'Es

In [75]:
descriptionsdf.T

,0,1,2,6,10,14,18,22,26,30,...,510,514,518,522,526,530,534,538,542,546
Column Name,GEO_ID,NAME,DP03_0001E,DP03_0002E,DP03_0003E,DP03_0004E,DP03_0005E,DP03_0006E,DP03_0007E,DP03_0008E,...,DP03_0128E,DP03_0129E,DP03_0130E,DP03_0131E,DP03_0132E,DP03_0133E,DP03_0134E,DP03_0135E,DP03_0136E,DP03_0137E
Label,Geography,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Estimate!!EMPLOYMENT STATUS!!In labor force,Estimate!!EMPLOYMENT STATUS!!In labor force!!C...,Estimate!!EMPLOYMENT STATUS!!In labor force!!C...,Estimate!!EMPLOYMENT STATUS!!In labor force!!C...,Estimate!!EMPLOYMENT STATUS!!In labor force!!A...,Estimate!!EMPLOYMENT STATUS!!Not in labor force,Estimate!!EMPLOYMENT STATUS!!Civilian labor force,...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...


In [76]:
descriptionsdf

,Column Name,Label
0,GEO_ID,Geography
1,NAME,Geographic Area Name
2,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
6,DP03_0002E,Estimate!!EMPLOYMENT STATUS!!In labor force
10,DP03_0003E,Estimate!!EMPLOYMENT STATUS!!In labor force!!C...
...,...,...
530,DP03_0133E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
534,DP03_0134E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
538,DP03_0135E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...
542,DP03_0136E,Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WH...


In [77]:
# List of descriptions to filter
desired_labels = [
    'Geography',
    'Geographic Area Name',
    'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over',
    'Estimate!!EMPLOYMENT STATUS!!In labor force',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Total households',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Mean household income (dollars)',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Families', 
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median family income (dollars)',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Mean family income (dollars)',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Per capita income (dollars)',
    'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median earnings for workers (dollars)',
    'Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families'
]

# Filter the DataFrame based on desired_labels
filtered_df = descriptionsdf[descriptionsdf['Label'].isin(desired_labels)]

# Print the filtered DataFrame
print(filtered_df)

    Column Name                                              Label
0        GEO_ID                                          Geography
1          NAME                               Geographic Area Name
2    DP03_0001E  Estimate!!EMPLOYMENT STATUS!!Population 16 yea...
6    DP03_0002E        Estimate!!EMPLOYMENT STATUS!!In labor force
42   DP03_0011E        Estimate!!EMPLOYMENT STATUS!!In labor force
202  DP03_0051E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
246  DP03_0062E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
250  DP03_0063E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
298  DP03_0075E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
345  DP03_0086E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
346  DP03_0087E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
350  DP03_0088E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
366  DP03_0092E  Estimate!!INCOME AND BENEFITS (IN 2010 INFLATI...
474  DP03_0119E  Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE W

In [78]:
small_label_list = list(filtered_df["Label"])
small_label_list

['Geography',
 'Geographic Area Name',
 'Estimate!!EMPLOYMENT STATUS!!Population 16 years and over',
 'Estimate!!EMPLOYMENT STATUS!!In labor force',
 'Estimate!!EMPLOYMENT STATUS!!In labor force',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Total households',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median household income (dollars)',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Mean household income (dollars)',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Families',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median family income (dollars)',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Mean family income (dollars)',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Per capita income (dollars)',
 'Estimate!!INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS)!!Median earnings for workers (dollars)',
 'Estimat